In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:###


#######################################################################################################################
##
## This is a utility script and contains all required function used by the tasks which to be ran in EC2 machine.
## This script calls config.py to get all required parameters.
## Created Date: 030-Mar-2020
## Version: 1.0
#####################################################################################################################

!pip install cx_Oracle
!pip install zipfile
!pip install docx2txt
!pip install boto3
!pip install extract_msg
!pip install PyPDF2
!pip install s3fs
!pip install bs4
!pip install docx
#!pip install BeautifulSoup
!pip install docx.api
!pip install Document
!pip install warnings
!pip install exceptions
!pip install html2text
!pip install O365
!pip install botocore


#import subprocess
#import ast
import json
#import fnmatch
import datetime
#import time
import re
import sys
import os
import cx_Oracle
import boto3
#import site
#from config_07_04_2020 import *
import linecache
import logging
import boto3
import psycopg2.extras
import psycopg2
import os
#import extract_msg
import pandas as pd
#import win32com.client
import datetime
from datetime import date
#import pywintypes
import re
import os
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import os
#import shutil
import zipfile
import docx2txt
#from DocumentExtract_v5 import *
import pandas as pd
import PyPDF2
import html2text
import datetime
import pytz
import base64
import botocore.exceptions
#import BeautifulSoup as bs4
#from docx.api import Document

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [2]:
####----------IMPORTS FOR ML MODEL --------#####################################

import itertools
import os
import re
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from tensorflow.keras.models import model_from_json
from keras import utils
from tensorflow.keras.models import load_model
#import keras
import boto3
import spacy
from keras import backend as K
import tensorflow as tf
print('You have TensorFlow version', tf.__version__)
import nltk
from sklearn.model_selection import train_test_split
nltk.download('punkt')
#nlp = spacy.load("/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/en_core_web_lg/en_core_web_lg-2.2.5")

from nltk.stem import WordNetLemmatizer, SnowballStemmer
#import gensim
#from gensim.parsing.preprocessing import STOPWORDS
#from gensim.test.utils import datapath
import matplotlib.pyplot as plt
import pickle
import re
#from bs4 import BeautifulSoup as bs4
from O365 import Account
#from bs4 import BeautifulSoup as bs4

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

Using TensorFlow backend.
You have TensorFlow version 2.1.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [3]:
#########################-------CONFIG FILE------##########################################
curpath='s3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Pipeline/Data_Pipeline/input/Full_Load_3.0/'
output_path = 's3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Pipeline/Data_Pipeline/output/Full_Load_3.0/'



###########################---------AUDIT BATCH Variables------------------#######################

global stage_list,milestone_list,starttime,endtime,runtime,processed_emails,batch_no,total_emails_in_batch,intake_list,process_list,log_list,email_triage_list,df_batch_log,total_emails_in_batch,df_input_to_process,total_emails_count

stage_list = []
milestone_list = []
starttime = []
endtime = []
runtime = []
processed_emails = []
batch_no = []
total_emails_in_batch = []
intake_list = []
process_list = []
log_list = []
email_triage_list = []
#stage_list,milestone_list,starttime,endtime,runtime,processed_emails,batch_no,total_emails_in_batch,intake_list,process_list,log_list,email_triage_list=([], )*12

total_emails_count=0
df_input_to_process=pd.DataFrame()
df_batch_log=pd.DataFrame()

intake_list=['get_emails_save_attachments_using_validation_sheet_via_windows_api','get_emails_save_attachments_within_a_timeframe_using_windows_api_batch']
process_list=['create_standard_dataframe','extract_from_attachment','create_ml_matrix_df','get_Model_output','get_mail_types']
log_list=['new_email_flatten','local_email_flatten','global_email_flatten','write_email_flattened_df']
email_triage_list=['email_triage']


########################Config Values for function get_subjectlist_from_validatn_sheet#####################
validatn_sheet_path = 's3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Pipeline/Data_Pipeline/input/Full_Load_3.0/'

validatn_sheet_name = 'export_905.csv'
#'check_200.csv'
#'export_full_925.csv'
#'Master_Validation_Data_AE_Final_15042020_Entyvio_CLN.csv'
#'Master_Validation_Data_AE_Final_15042020_Entyvio.csv'
#'entivyo_check.csv'
#'check.csv'
#'df_300_run'
#'dup_check.csv'
#'Validation_Data_AE_Final_15042020.csv'
#'dup_check.csv'
#'check_for_attchmnt.csv'
#'validation_input_sheet.csv'
#'validation_input_sheet_500.csv'

pattern =  r"TAKA\s\d{2}\s\d{5}\s[A-Za-z]{3}\d{4}"



##################------Config for function get_emails_save_attachments_using_windows_api--------#################
v_resource = 'smb.Dso-Ca-Tide-Pilot@takeda.com'
path_of_attachment = 'rnd-us-raw/tide/Pipeline/Exports/'
pipeline_input_csv = 'export_905.csv'
pipeline_input_json = 'export.json'


##################---------Config for function upload_file_s3--------##############
bucket = 'tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'

##################---------Config for function get_attachment_contents--------##############
checklist = ['FORM-0003904', 'G-Form-PV-050', 'G-Form-PV-054', 'G-Form-PV-075', 'G-FORM-PV-0751o']
form_050='G-Form-PV-050'
form_075='G-Form-PV-075'
form_054='G-Form-PV-054'
form_3904='FORM-0003904'


##################---------Config for function get_mail_types--------##############
AEThreshold_ConfScore = 0.85
SSThreshold_ConfScore = 0.85
OtherThreshold_ConfScore = 0.85
GlobalThreshold_ConfScore = 0.85

local_pattern = r"CA-\d{4}"
global_id_pattern = r"\d{4}[A-Z]{3}\d{6}"
local_id_sender = r"DSO-CA <DSO-CA@takeda.com>"

######################--------Variables For ML Code----------###################
warning_msg_file = 's3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Data Preparation/warning_messages.csv'
custom_stopwords_file = 's3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Data Preparation/custom_stopwords.csv'


###Global Variables
bucketname='tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'
Tokenizerpath='/rnd-us-raw/tide/Model Building/tokenizer_23042020_AE.pickle'
ModelPath='/rnd-us-raw/tide/Model Building/model_23042020_DNN_AE.h5'

threshold=0.5
vocab_special=['gestational','gestation','pregnancy','maternity','post-delivery','off label combination','off label regimen','off label',
'off-label','Off label use',
'off-label use','Off-label use','Off-label ','disease is progressing','Progresssive disease','disease appears to be progressing',
'disease is improving','lack of effect','disease progression','wrong information','overdose','overdosed']

###Global Variables
bucketname='tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'
Tokenizerpath_fatal='rnd-us-raw/tide/Model Building/DNN/tokenizer_25042020_Fatal_2.pickle'
ModelPath_fatal='rnd-us-raw/tide/Model Building/DNN/model_25042020_DNN_Fatal_2.h5'

###Global Variables
bucketname='tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'
Tokenizerpath_serious='rnd-us-raw/tide/Model Building/DNN/tokenizer_27042020_Serious_Nonserious.pickle'
ModelPath_serious='rnd-us-raw/tide/Model Building/DNN/model_27042020_Serious_1.h5'


######################--------Variables 1 hour batch----------###################
###Global Variables

Mails_Recieved_After = '2019-11-17 06:00:00'
Time_Window_min = 20000
max_no_of_emails_in_api_batch = 20




In [4]:

#bucketname='tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'
#Tokenizerpath='/rnd-us-raw/tide/Model Building/tokenizer_23042020_AE.pickle'
#ModelPath='/rnd-us-raw/tide/Model Building/model_23042020_DNN_AE.h5'
#
#def get_model_token_DNN_AE(bucketname,key,modelkey):
#  s3client = boto3.client('s3')
#  response = s3client.get_object(Bucket=bucketname, Key=key)
#  tokenizer_load = response['Body'].read()
#  tokenizer_load = pickle.loads(tokenizer_load)
#
#  result = s3client.download_file(bucketname,modelkey, "model_23042020.h5")
#  model = load_model("model_23042020.h5",custom_objects={'f1_m':f1_m,'recall_m':recall_m,'precision_m':precision_m})
#  
#  return tokenizer_load,model
#
#tokenizer,model=get_model_token_DNN_AE(bucketname,Tokenizerpath,ModelPath)

In [5]:
def get_keys_from_secret_store_manager():
    secret_name = "OutlookAPI"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            print('reached 1')
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            print('reached 2')
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            print('reached 3')
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            print('reached 4')
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            print('reached 5')
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        else:
            print('reached last')
            # Decrypts secret using the associated KMS CMK.
            # Depending on whether the secret is a string or binary, one of these fields will be populated.
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
        secret = json.loads(secret)
        return secret
    else:
        decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
        return decoded_binary_secret
    

In [6]:
#df_736 = pd.read_csv(curpath+'df_736.csv')
#df_400 = pd.read_csv(curpath+'df_400.csv')
#all_df1_ids = df_400.src_id.values.tolist()
#df_300 = df_736[df_736.src_id.apply(lambda src_id : src_id not in all_df1_ids)]
#df_300.to_csv(curpath+'df_300.csv')
#import re
#ptrn = re.compile("<([^>]+)>")
#email = '<Nilesh.Utpure@cognizant.com>'
#email1 = re.findall(ptrn,email)
#
#    
#print(email1[0])



In [7]:
##############################-------- AUDIT BATCH LOG FUNCTIONS -----------#####################

def check_Audit_Log():
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('tpc-aws-ted-dev-edpp-raw-rnd-us-east-1')
    # output_path = 'rnd-us-raw/tide/Pipeline/Data_Pipeline/output/Full_Load/'
    key = output_path + 'Audit_Batch_Log_Master.csv'
    try:
        df_check = pd.read_csv(key)
        print("Exists!")
        return True
    except:
        print("Exists!")
        return False
    # objs = list(bucket.objects.filter(Prefix=key))
    # if len(objs) > 0 and objs[0].key == key:
    #    print("Exists!")
    #    return True
    # else:
    #    print("Doesn't exist")
    #    return False


def get_max_batchid():
    y = check_Audit_Log()
    if (not y):
        return 0
    else:
        df_batch = pd.read_csv(output_path + 'Audit_Batch_Log_Master.csv')
        df_max1 = df_batch[df_batch['Batch_Id'] == df_batch['Batch_Id'].max()]
        df_max = df_max1.reset_index()
        max_batchid = df_max['Batch_Id'][0]
        return (max_batchid)

def get_maximum_received_datetime():    
    df_validation_sheet = pd.read_csv(validatn_sheet_path+validatn_sheet_name)
    df_validation_sheet['ReceivedDate'] = df_validation_sheet['ReceivedDate'].apply(lambda pd_time : convert_into_datetime(pd_time))
    max_received_datetime = df_validation_sheet['ReceivedDate'].max().to_pydatetime()
    return max_received_datetime      

def insert_into_batchlog():
    y = check_Audit_Log()
    dateTimeObj = str(datetime.datetime.now())
    df_batch = pd.DataFrame()
    df_final = pd.DataFrame()
    if y:
        df_batch_hist = pd.read_csv(output_path + 'Audit_Batch_Log_Master.csv')
        df_final = df_final.append(df_batch_hist)
    max_batchid1 = get_max_batchid()
    print(max_batchid1)
    max_batchid = int(max_batchid1) + 1
    if (max_batchid1 == 0):
        df_batch.loc[0, 'Batch_Id'] = 1
        df_batch.loc[0, 'Total_Emails_In_Batch'] = None
        df_batch.loc[0, 'Batch_Start_Time'] = dateTimeObj
        df_batch.loc[0, 'Batch_End_Time'] = None
        df_batch.loc[0, 'Batch_Status'] = 'Running'
        df_batch.loc[0, 'Max_ReceivedTime_Of_Email'] = None

    else:        
        df_batch.loc[0, 'Batch_Id'] = max_batchid
        df_batch.loc[0, 'Batch_Start_Time'] = dateTimeObj
        df_batch.loc[0, 'Batch_End_Time'] = None
        df_batch.loc[0, 'Batch_Status'] = 'Running'
        df_batch.loc[0, 'Max_ReceivedTime_Of_Email'] = None

    print('Audit_Batch_Log entry generated!!!')
    df_final = df_final.append(df_batch)
    df_final.reset_index(inplace=True, drop=True)
    batch_id = df_final['Batch_Id'].max()
    df_final = df_final[['Batch_Id','Total_Emails_In_Batch','Batch_Start_Time','Batch_End_Time','Max_ReceivedTime_Of_Email','Batch_Status']]
    df_final.to_csv(output_path + 'Audit_Batch_Log_Master.csv', index=False)
    return df_final, batch_id


def update_into_batchlog(status, total_emails_in_batch):
    max_received_datetime = get_maximum_received_datetime()
    dateTimeObj = str(datetime.datetime.now())
    if total_emails_in_batch == 0:
        total_emails_in_batch == 'Data Ingestion Step Failed'

    df1 = pd.read_csv(output_path + 'Audit_Batch_Log_Master.csv')
    df_final = pd.DataFrame()
    max_batchid1 = get_max_batchid()
    df2 = df1[df1['Batch_Id'] == max_batchid1]
    df_max = df2.reset_index(drop=True)
    df_hist = df1[df1['Batch_Id'] != max_batchid1]
    df_hist.reset_index(inplace=True, drop=True)
    df_final = df_final.append(df_hist)
    df_final.reset_index(inplace=True, drop=True)
    df_max.loc[0, 'Batch_End_Time'] = dateTimeObj
    df_max.loc[0, 'Batch_Status'] = status
    df_max.loc[0, 'Total_Emails_In_Batch'] = total_emails_in_batch
    df_max.loc[0, 'Max_ReceivedTime_Of_Email'] = max_received_datetime
    df_final = df_final.append(df_max)
    df_final.reset_index(inplace=True, drop=True)
    df_final = df_final[['Batch_Id','Total_Emails_In_Batch','Batch_Start_Time','Batch_End_Time','Max_ReceivedTime_Of_Email','Batch_Status']]
    df_final.to_csv(output_path + 'Audit_Batch_Log_Master.csv', index=False)
    return df_final


def audit_batch_log_detail(batch_id, total_emails_count, df_current, milestone, status):
    t = datetime.datetime.now()
    #print('*****ENTERED DETAIL LOG ********')
    # count = int(df.shape[0])

    if status == 'start':
        milestone_list.append(milestone)
        starttime.append(t)
        batch_no.append(batch_id)
        total_emails_in_batch.append(None)
        if milestone in intake_list:
            stage_list.append('Intake')
        elif milestone in process_list:
            stage_list.append('Processing')
        elif milestone in log_list:
            stage_list.append('Logging')
        elif milestone in email_triage_list:
            stage_list.append('Triaging of emails')
        endtime.append('milestone not completed yet')
        runtime.append('milestone not completed yet')
        processed_emails.append('milestone not completed yet')
        #print('********** INSIDE START DETAIL*********')
        #print('batch_no', batch_no, 'milestone_list', milestone_list, 'starttime', starttime, 'stage_list', stage_list)
    elif status == 'end':
        count = df_current.shape[0]
        endtime[-1] = t
        t1 = starttime[-1]
        t2 = endtime[-1]
        t3 = (t2 - t1).total_seconds()
        runtime[-1] = t3
        processed_emails[-1] = count
        total_emails_in_batch[-1] = total_emails_count
        #print('********** INSIDE END DETAIL*********')
        #print('batch_no', batch_id, batch_no, 'milestone_list', milestone_list, 'starttime', starttime, 'stage_list',
              #stage_list, 'endtime', endtime, 'processed_emails', processed_emails)
    data_tuple = list(
        zip(batch_no, total_emails_in_batch, processed_emails, stage_list, milestone_list, starttime, endtime, runtime))
    df_batch_log=pd.DataFrame(data_tuple,columns=['Batch_Id','Total_Emails_In_Batch','Processed_Emails','Stage','Milestone','Starttime','Endtime','Runtime'])

    # print(df_runtime)
    #print('**** OUTSIDE DETAIL LOOP')
    #print('batch_no', batch_no, 'total_emails_in_batch', total_emails_in_batch)
    #df_batch_log = pd.DataFrame(data_tuple,
    #                            columns=['batch_no', 'total_emails_in_batch', 'processed_emails', 'stage_list',
    #                                     'milestone_list', 'starttime', 'endtime', 'runtime'])
    df_batch_log.drop_duplicates(subset = ['Milestone'], keep = 'last', inplace = True)
    df_batch_log.reset_index(inplace=True,drop=True)
    if batch_id == 1:
        df_batch_log.to_csv(output_path + 'Audit_Batch_Log_Detail.csv')
    elif batch_id != 1:
        df_batch_log.to_csv(output_path + 'Audit_Batch_Log_Detail.csv', mode='a', header=False)
    return df_batch_log

In [8]:
def oulook_access_authentication():
    # Accessing outlook usering API
    key_val_dict = {}
    key_val_dict = get_keys_from_secret_store_manager()
    access_id = key_val_dict['Application_ID']
    key = key_val_dict['key']
    v_tenant_id = key_val_dict['Tenant_ID']
    credentials = (access_id, key)
    account = Account(credentials, auth_flow_type='credentials', tenant_id=v_tenant_id)
    if account.authenticate():
        print('Authenticated!')
    return account


def upload_file_s3(filename,attch_name):      
    import boto3
    s3 = boto3.resource('s3')
    s3.Bucket('tpc-aws-ted-dev-edpp-raw-rnd-us-east-1').upload_file(filename,path_of_attachment + attch_name)

# def upload_file_s3(filename):
#   import boto3
#   s3 = boto3.resource('s3')
#   s3.Bucket('tpc-aws-ted-dev-edpp-raw-rnd-us-east-1').upload_file(filename,'rnd-us-raw/tide/Pipeline/Exports/'+filename)

def add_minutes_to_time(pd_timestamp):
    python_time = pd.to_datetime(pd_timestamp)
    python_time = python_time.to_pydatetime()
    python_time = python_time.replace(second=0, tzinfo=None)
    python_time = python_time + datetime.timedelta(minutes=330)
    return python_time
  
def convert_into_datetime(pd_timestamp):
    python_time = pd.to_datetime(pd_timestamp)
    python_time = python_time.to_pydatetime()
    python_time = python_time.replace(second=0, tzinfo=None)
    #python_time = python_time + datetime.timedelta(minutes=330)
    return python_time
  
def clean_sender(ip_value):
    junk = ['"','<','>','(',')']
    for item in junk:
        ip_value = ip_value.replace(item, '')
    return ip_value
  
  
def roundTime(dt):    
    """Round a datetime object to any time lapse in seconds
    dt : datetime.datetime object, default now.
    roundTo : Closest number of seconds to round to, default 1 minute.
    Author: Thierry Husson 2012 - Use it as you want but don't blame me.
    """  
    roundTo=60
    seconds = (dt.replace(tzinfo=None) - dt.min).seconds
    rounding = (seconds+roundTo/2) // roundTo * roundTo          
    return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)
  


  
def get_data_from_validatn_sheet(validatn_sheet_path,validatn_sheet_name):
    # loading validation data
    df_validation_sheet = pd.read_csv(validatn_sheet_path+validatn_sheet_name)
    df_validation_sheet['windowsAPI_extraction_flag'] = 'N'
    df_validation_sheet['src_id'] = range(0,df_validation_sheet.shape[0])
    df_validation_sheet['Date'] = df_validation_sheet['Date'].apply(lambda pd_time : convert_into_datetime(pd_time))
    df_validation_sheet['Subject_Match_Status'] = 'Failed'
    df_validation_sheet['Sender_Match_Status'] = 'Failed'
    df_validation_sheet['Time_Match_Status'] = 'Failed'
    df_validation_sheet['subj_frm_sheet'] = ''
    df_validation_sheet['subj_frm_api'] = ''
    df_validation_sheet['sender_frm_sheet'] = ''
    df_validation_sheet['sender_frm_api'] = ''
    df_validation_sheet['time_frm_sheet'] = ''
    df_validation_sheet['time_frm_api'] = ''
    df_validation_sheet['api_time_five_min_add'] = ''
    df_validation_sheet['api_time_five_min_sub'] = ''
    df_validation_sheet['email_body_frm_sheet'] = df_validation_sheet['mail']
    df_validation_sheet['email_body_frm_api'] = ''
    
                    
    #df_file_name = pd.DataFrame(df_validation_sheet['File Name'])
    #df_file_name.head()
    #df_subject = pd.DataFrame(df_validation_sheet['Subject'])
    #pattern = r"TAKA\s\d{2}\s\d{5}\s[A-Za-z]{3}\d{4}"
    #subject_list = []
    #subject_list = df_Val['Subject'].tolist()
    return df_validation_sheet #subject_list
                                
def get_emails_save_attachments_using_validation_sheet_via_windows_api(batch_id,validatn_sheet_path,validatn_sheet_name):
    account = oulook_access_authentication()
    mailbox = account.mailbox(resource=v_resource)
    df_validation_sheet = get_data_from_validatn_sheet(validatn_sheet_path,validatn_sheet_name)
    #mailbox = mailbox.get_folder(folder_name='DSO-CA-Test')
    #child_folders = archive.get_folders(25)
    #print(child_folders)
    #for folder in child_folders:
        #if folder.name == 'DSO-CA-Test':                                      
    #mailbox = account.mailbox()
    #archive = mailbox.get_folder(folder_name='COPIED-DSO-CA@takeda.com')
    #test_mailbox = mailbox.get_folder(folder_name="COPIED-DSO-CA@takeda.com")
    #query = test_mailbox.new_query().on_attribute('subject').contains("my_subject")

    #subject_list = get_subjectlist_from_validatn_sheet(validatn_sheet_path,validatn_sheet_name)
    #print(subject_list)
    my_list = []
    count = int(df_validation_sheet.shape[0])
    print('Searching for specific emails from Mailbox')
    for i in range(count):
        print('inside loop')
        subj_frm_sheet = str(df_validation_sheet['Subject'][i]).strip()  #---P
        #receiver = str(df_validation_sheet['To'][i]).strip()
        print('working with Subject ',subj_frm_sheet)
        query = mailbox.new_query().on_attribute('subject').contains(subj_frm_sheet)  # see Query object in Utils
        try:          
            messages = mailbox.get_messages(query=query, download_attachments=True)  #limit=10, 
            subj_frm_sheet = subj_frm_sheet.upper()
            ptrn = re.compile("<([^>]+)>")
            sender_frm_sheet = str(df_validation_sheet['From'][i])
            sender_frm_sheet_lst = re.findall(ptrn,sender_frm_sheet)
            if sender_frm_sheet_lst:       #---P                                          
                sender_frm_sheet = sender_frm_sheet_lst[0]
            
            sender_frm_sheet = sender_frm_sheet.strip().upper()
            #sender_frm_sheet = clean_sender(sender_frm_sheet)
            if sender_frm_sheet == 'DSO-CA':
                sender_frm_sheet = 'DSO-CA@TAKEDA.COM'
            time_frm_sheet = df_validation_sheet['Date'][i].to_pydatetime()
            ENtyvio = df_validation_sheet['ENtyvio'][i]
            #print('working with Subject ',subj_frm_sheet)    
            #print(messages)
            for msg in messages:            
                src_link_id = df_validation_sheet['src_id'][i]                
                msg_id = msg.internet_message_id
                subj_frm_api = msg.subject
                subj_frm_api = str(subj_frm_api).strip().upper()
                sender_frm_api = msg.sender.address 
                sender_frm_api = str(sender_frm_api).strip().upper()
                #sender_frm_api = clean_sender(sender_frm_api)
                time_frm_api = msg.received
                time_frm_api = str(time_frm_api)
                time_frm_api = add_minutes_to_time(time_frm_api)
                email_body_frm_api = html2text.html2text(msg.body)
                api_time_five_min_sub = time_frm_api + datetime.timedelta(minutes=-30)
                api_time_five_min_add = time_frm_api + datetime.timedelta(minutes=30)
                list = []
                ##print('loop inside, just before duplicate check')
                #print('subj_frm_sheet: ',subj_frm_sheet,' subj_frm_api: ',subj_frm_api)
                ##print('sender_frm_sheet: ',sender_frm_sheet,' sender_frm_api: ',sender_frm_api)
                ##print('time_frm_sheet: ',time_frm_sheet,' time_frm_api: ',time_frm_api)
                ##print('email_body_frm_api: ',email_body_frm_api)
                ##print('api_time_five_min_sub: ',api_time_five_min_sub)
                ##print('api_time_five_min_add',api_time_five_min_add)
                #print((subj_frm_sheet==subj_frm_api and sender_frm_sheet==sender_frm_api and time_frm_sheet==time_frm_api))
                #print((subj_frm_sheet==subj_frm_api and sender_frm_sheet==sender_frm_api and time_frm_sheet==api_time_one_min_sub))
                #print('***************Status of exact SUBJECT match*******************',subj_frm_sheet==subj_frm_api)
                #print('**************Status of exact SENDER match***********',sender_frm_sheet==sender_frm_api)
                #print('*************Status of  exact SUBJECT match**************',time_frm_sheet==time_frm_api)
                df_validation_sheet['subj_frm_sheet'][i] = subj_frm_sheet
                df_validation_sheet['subj_frm_api'][i] = subj_frm_api
                df_validation_sheet['sender_frm_sheet'][i] = sender_frm_sheet
                df_validation_sheet['sender_frm_api'][i] = sender_frm_api
                df_validation_sheet['time_frm_sheet'][i] = time_frm_sheet
                df_validation_sheet['time_frm_api'][i] = time_frm_api
                df_validation_sheet['email_body_frm_api'][i] = email_body_frm_api
                df_validation_sheet['api_time_five_min_add'][i] = api_time_five_min_add
                df_validation_sheet['api_time_five_min_sub'][i] = api_time_five_min_sub
                
                
                if (subj_frm_sheet==subj_frm_api):
                    df_validation_sheet['Subject_Match_Status'][i] = 'Passed'
                                     
                if (sender_frm_sheet==sender_frm_api):
                    df_validation_sheet['Sender_Match_Status'][i] = 'Passed'
                    
                if (time_frm_sheet>=api_time_five_min_sub and time_frm_sheet<=api_time_five_min_add):
                    df_validation_sheet['Time_Match_Status'][i] = 'Passed'
                                                        
                                      
                if ((ENtyvio != 'Y' and subj_frm_sheet==subj_frm_api and sender_frm_sheet==sender_frm_api and time_frm_sheet>=api_time_five_min_sub and time_frm_sheet<=api_time_five_min_add) or (ENtyvio == 'Y')):
                    print('***********************DID IT*******************************')
                    df_validation_sheet['windowsAPI_extraction_flag'][i] = 'Y'
                    
                    if msg.has_attachments:
                        attch_id = 1
                        for att in msg.attachments:
                            print('******************entered attachment loop*************************')
                            print(attch_id)
                            print('check1')
                            print(len(msg.attachments))
                            filename = att.name
                            attch_name = str(attch_id) + '__' + msg_id + '__' + filename
                            list.append(attch_name)
                            print(list)
                            att.save()
                            print('saved')
                            upload_file_s3(filename,attch_name)                        
                            print('check3')
                            attch_id = attch_id + 1

                    print(msg.subject)
                    my_list.append(
                        [src_link_id, sender_frm_api, msg.to, roundTime(time_frm_api), msg.subject, html2text.html2text(msg.body), msg_id,
                         len(msg.attachments), path_of_attachment, list])
                    break;
        except: 
            print('error')            
    
    global df
    df_validation_sheet.to_csv(curpath+'flagged_validation_sheet.csv')
    print('***********************CAME HERE*******************************')
    df_input_to_process = pd.DataFrame(my_list,
                      columns=['src_link_id','From', 'To', 'ReceivedDate', 'Subject', 'MailBody_Text', 'message_id',
                               'no_of_attachments',
                               'path_of_attachement', 'attachment_names'])
    # print(len(msg.attachments))
    #df_input_to_process.drop_duplicates(subset=['From','To','Subject','no_of_attachments','path_of_attachement'],keep='first',inplace = True)
    df_input_to_process.drop_duplicates(subset = ['From','MailBody_Text','ReceivedDate'], keep = 'first', inplace = True)
    df_input_to_process.drop_duplicates(subset = ['src_link_id','From','MailBody_Text'], keep = 'first', inplace = True)
    df_input_to_process.reset_index(inplace=True,drop=True)
    df_input_to_process['batch_id'] = batch_id
    print('***********************CAME HERE2*******************************')
    df_input_to_process.to_csv(curpath+pipeline_input_csv)
    #df_input_to_process.to_json(curpath+pipeline_input_json)
    #upload_file_s3(curpath+pipeline_input_csv)
    #upload_file_s3(curpath+pipeline_input_json)
    return df_input_to_process,df_validation_sheet


In [9]:

def add_min_to_time(time_string,no_of_min):
    python_time = pd.to_datetime(time_string)
    python_time = python_time.to_pydatetime()
    python_time = python_time.replace(second=0, tzinfo=None)
    python_time = python_time + datetime.timedelta(minutes=no_of_min)
    return python_time

def get_emails_save_attachments_within_a_timeframe_using_windows_api_batch(batch_id,Start_Date_Time,Time_Window_Min):
    #import dt
    account = oulook_access_authentication()
    mailbox = account.mailbox(resource=v_resource)
    my_list = []
    Start_Date_Time = convert_into_datetime(Start_Date_Time)    
    Start_Year = Start_Date_Time.year  
    End_Date_Time = add_min_to_time (Start_Date_Time,Time_Window_Min)  
    query = mailbox.new_query().on_attribute('ReceivedDateTime').less_equal(End_Date_Time).chain('and').on_attribute('ReceivedDateTime').greater(Start_Date_Time)
            
    try:          
        messages = mailbox.get_messages(query=query, download_attachments=True) #limit=max_no_of_emails_in_api_batch,

        for msg in messages:
            msg_id = msg.internet_message_id
            list = []
            if msg.has_attachments:
              attch_id = 1
              for att in msg.attachments:
                filename = att.name
                attch_name = str(attch_id) + '__' + msg_id + '__' + filename
                list.append(attch_name)
                att.save()
                upload_file_s3(filename,attch_name)                        
                attch_id = attch_id + 1
            my_list.append(
                ['NULL', msg.sender.address, msg.to, roundTime(msg.received), msg.subject, html2text.html2text(msg.body), msg_id,
                 len(msg.attachments), path_of_attachment, list])
    except:        
        print('error')            

    
    df_input_to_process = pd.DataFrame(my_list,
                      columns=['src_link_id','From', 'To', 'ReceivedDate', 'Subject', 'MailBody_Text', 'message_id',
                               'no_of_attachments',
                               'path_of_attachement', 'attachment_names'])
    # print(len(msg.attachments))
    #df_input_to_process.drop_duplicates(subset=['From','To','Subject','no_of_attachments','path_of_attachement'],keep='first',inplace = True)
    df_input_to_process.drop_duplicates(subset = ['From','MailBody_Text','ReceivedDate'], keep = 'first', inplace = True)
    
    df_input_to_process['batch_id'] = batch_id
    df_input_to_process.to_csv(curpath+pipeline_input_csv,index=False)
    return df_input_to_process


In [10]:

###########--------DocumentExtract_v5.py

def process_text(file_name):
    text = docx2txt.process(file_name)
    text = re.sub(r"[\n\t\u2002\xa0]", " ", text)
    return text


def get_form_number(text):
    form_number = re.search(r"(Form Number:\s(.*)Page:)", text).group(0)
    form_number = re.sub(r"(Form Number:)|(Page:)|\s", '', form_number)
    return form_number

def output_df(formNum,LocalID,Narrative,SusProd,Vendor,ReName,PatientInitial,PatientAge,PatientID,Sol):
    dict_extract = {'column':[],'value':[]}
    column=['Form Name','Local Case ID','Narrative','Suspect Taked Product','Vendor Name','Reporter Name/Initials',
            'Patient Initial','Patient Age','Patient ID','solicited/unsolicited']
    value=[formNum,LocalID,Narrative,SusProd,Vendor,ReName,PatientInitial,PatientAge,PatientID,Sol]
    dict_extract['column']=column
    dict_extract['value']=value
    
    return pd.DataFrame(dict_extract)


# In[12]:


## AE Local PV Form 0003904

def get_TradeGenDrug_0003904(text):
    if re.search(r"(Trade name\s(.*)Takeda product )", text): 
        TradeName = re.search(r"(Trade name\s(.*)Takeda product )", text).group(0)
        TradeName = re.sub(r"(Trade name)|(Takeda product )|\s", '', TradeName)
    else:
        TradeName=''
    if re.search(r"(Generic Name\s(.*)Indication )", text): 
        GenericName = re.search(r"(Generic Name\s(.*)Indication )", text).group(0)
        GenericName = re.sub(r"(Generic Name)|(Indication )|\s", '', GenericName)
    else:
        GenericName=''
    return TradeName,GenericName

def get_VendorReporter_0003904(text):
    if re.search(r"(Vendor Case ID:(.*)Program Name  )", text):
        VendorID = re.search(r"(Vendor Case ID:(.*)Program Name  )", text).group(0)
        VendorID = re.sub(r"(Vendor Case ID:)|(Program Name )|\s", '', VendorID)
    else:
        VendorID=''
    
    Reporter=re.sub(r"[()]","",text)
    if re.search(r"(Initials/Name  First/Last  Media site Name (.*)Postal Address  Country)", Reporter):
        Reporter = re.search(r"(Initials/Name  First/Last  Media site Name (.*)Postal Address  Country)", Reporter).group(0)
        Reporter = re.sub(r"(Initials/Name  First/Last  Media site Name)|(Postal Address  Country)|\s", '', Reporter)
    else:
        Reporter=''
    
    return VendorID,Reporter
        
def get_Patient_0003904(text):
    if re.search(r"(Initials (.*)Age  )", text):
        PInitial = re.search(r"(Initials (.*)Age  )", text).group(0)
        PInitial = re.sub(r"(Initials )|(Age  )|\s", '', PInitial)
    else:
        PInitial=''
    if re.search(r"(Age (.*)Date of Birth  )", text):
        PAge = re.search(r"(Age (.*)Date of Birth  )", text).group(0)
        PAge = re.sub(r"(Age )|(Date of Birth  )|\s", '', PAge)
    else:
        PAge=''
    if re.search(r"(Date of Birth  (.*)Ethnicity )", text):
        PDob = re.search(r"(Date of Birth  (.*)Ethnicity )", text).group(0)
        PDob = re.sub(r"(Date of Birth  )|(Ethnicity )|\s", '', PDob)
    else:
        PDob=''
    if re.search(r"(Gender  (.*)Pregnant:  )", text):
        Pgender = re.search(r"(Gender  (.*)Pregnant:  )", text).group(0)
        Pgender = re.sub(r"(Gender  )|(Pregnant: )|\s", '', Pgender)
    else:
        Pgender=''
    return PInitial,PAge,PDob,Pgender
    

def get_Event_Verbatim_0003904(text):
    narrative = re.search(r"(reporter causality, seriousness etc.(.*)Complete all fields where information is available)", text)[0]
    narrative = re.sub(r"(reporter causality, seriousness etc.:|Complete all fields where information is available|/s)",'', narrative)
    return narrative

def get_Event_Term_0003904(text):
    narrative = re.search(r"(reporter causality, seriousness etc.(.*)Complete all fields where information is available)", text)[0]
    narrative = re.sub(r"(reporter causality, seriousness etc.:|Complete all fields where information is available|/s)",'', narrative)
    return narrative

def get_narrative_0003904(text):
    narrative = ''
    return narrative


# In[13]:


## HCP Form PV-054

def get_Drug_054(document,tableid):
    table = document.tables[tableid]
    Trade_Name=re.search(r"(Suspected Drug:\s(.*)Indication):", table.cell(1,0).text)[0]
    Trade_Name = re.sub(r"(Suspected Drug:)|(Indication:)|\s|\t", '', Trade_Name)
    return Trade_Name

def get_Reporter_054(text):
    Reporter = re.search(r"(Reporter  Name:\s(.*)Profession)", text)[0]
    Reporter = re.sub(r"(Reporter  Name:)|(Profession)|\s", '', Reporter)
    return Reporter

def get_patientdetails_054(text):
    initial = re.search(r"(Patient Initials:\s(_.*)Gender)", text)[0]
    initial = re.sub(r"(Patient Initials:)|(Gender)|\s|_", '', initial)

    Age = re.search(r"(Age(.*)Height:)", text)[0]
    Age = re.sub(r"[Age (y/m/d): __]|Height","",Age)
    
    return initial,Age

def get_Narrative_054(text):
    narrative = re.search(r"(more than 2 reactions(.*)Date of onset)", text)[0]
    narrative = re.sub(r"(more than 2 reactions|Date of onset|/s)",'', narrative)
    return narrative


# In[14]:


### Form PV-050
def get_Drug_050(text):
    if re.search(r"(Trade Name(.*)Takeda Product?)", text):
        TradeName = re.search(r"(Trade Name(.*)Takeda Product)", text)[0]
        TradeName = re.sub(r"(Trade Name)|\s", '', TradeName.split('Takeda Product')[0])
    else:
        TradeName=''
    if re.search(r"(Generic Name(.*)Off label use?)", text):
        genericName = re.search(r"(Generic Name(.*)Indication  )", text)[0]
        genericName = re.sub(r"(Generic Name)|(Indication)|(Off label use)|\s|_", '', genericName.split('Indication')[0])
    else:
        genericName=''
    return TradeName,genericName


def get_ReporterVendor_050(text):
    VendorID='' ## Vendor ID field is not present in the document
    if re.search(r"(Name(.*)Primary Reporter?)", text):
        reporter = re.search(r"(Name(.*)Primary Reporter?)", text)[0]
        reporter = re.sub(r"(Name|(First/Last))|(Primary Reporter)|\s|_", '', reporter)
        reporter = re.sub(r"[()]","",reporter)
    else:
        reporter=''
           
    return reporter,VendorID

def get_patientdetails_050(text):
    PInitial='' ##Patient Initial field is not present in the document
    #PID = re.search(r"(Patient ID(.*)Gender)", text)[0]
    #PID = re.sub(r"(Patient ID|Gender|\s)",'', PID)
    if re.search(r"(Age  (.*)Weight)", text):
        PAge = re.search(r"(Age  (.*)Weight)", text)[0]
        PAge = re.sub(r"(Age  |Weight|\s)",'', PAge)
    else:
        PAge='' 
    if re.search(r"(Date of Birth  (.*)Ethnicity  )", text):
        PDob = re.search(r"(Date of Birth  (.*)Ethnicity  )", text)[0]
        PDob = re.sub(r"(Date of Birth  |Ethnicity  |\s)",'', PDob)
    else:
        PDob=''
    if re.search(r"(Gender  (.*)Pregnant:  )", text):
        PGender = re.search(r"(Gender  (.*)Pregnant:  )", text)[0]
        PGender = re.sub(r"(Gender  |Pregnant:  |\s)",'', PGender)
    else:
        PGender=''
    
    
    return PInitial,PAge,PDob,PGender



def get_Narrative_050(text):
    
    try:
        Narrative_New = re.search(r"(Narrative(.*)Comment)", text)[0]
        Narrative_New = re.sub(r"(Narrative|Comment|/s)",'', Narrative_New)
        
    except:
        Narrative_New=''
        pass

    return Narrative_New



def get_Event_Verbatim_050(text):

    try:
        Narrative = re.search(r"(Event Verbatim(.*)Seriousness Criteria)", text)[0]
        Narrative = re.sub(r"(Event Verbatim|Seriousness Criteria)",'', Narrative)
    except:
        Narrative=''
        pass
    
    return Narrative


def get_Event_Term_050(text):

    try:
        Narrative = re.search(r"(Event Verbatim(.*)Seriousness Criteria)", text)[0]
        Narrative = re.sub(r"(Event Verbatim|Seriousness Criteria)",'', Narrative)
    except:
        Narrative=''
        pass
    try:
        Narrative_New = re.search(r"(Narrative(.*)Comment)", text)[0]
        Narrative_New = re.sub(r"(Narrative|Comment|/s)",'', Narrative_New)
        
    except:
        Narrative_New=''
        pass

    return Narrative+Narrative_New



# In[15]:


## PV 0075
def get_drugNameVendor_0075(text):
    if re.search(r"(Trade Name:(.*)Vendor Case)", text):
        TradeName = re.search(r"(Trade Name:(.*)Vendor Case)", text)[0]
        TradeName = re.sub(r"(Trade Name:|Vendor Case|/s)",'', TradeName)
        
    elif re.search(r"(Trade name(.*)Generic Name)", text):
        TradeName = re.search(r"(Trade name(.*)Generic Name)", text)[1]
        TradeName=TradeName[10:22]
    else:
        TradeName=''

    if re.search(r"(Generic Name:(.*)Vendor Name:)", text):
        GenericName = re.search(r"(Generic Name:(.*)Vendor Name:)", text)[0]
        GenericName = re.sub(r"(Generic Name:|Vendor Name:|/s)",'', GenericName)
    elif re.search(r"(Generic Name(.*)Formulation  )", text):
        GenericName = re.search(r"(Generic Name(.*)Formulation  )", text)[0]
        GenericName=GenericName[12:24]
    else:
        GenericName=''
        
    if re.search(r"(Vendor Case/Ref ID:(.*)Vendor Name:  )", text):
        VendorID = re.search(r"(Vendor Case/Ref ID:(.*)Vendor Name:  )", text)[0]
        VendorID = re.sub(r"(Vendor Case/Ref ID:|/s)",'', VendorID.split('Generic Name')[0])
    elif re.search(r"(Vendor Case ID:(.*)Program Name  )", text):
        VendorID = re.search(r"(Vendor Case ID:(.*)Program Name  )", text)[0]
        VendorID = re.sub(r"(Vendor Case ID:|Program Name  |/s)",'', VendorID)
    else:
        vendorID=''
    
    return TradeName,GenericName,VendorID

def get_Reporter_075(text):
    Reporter=re.sub(r"[()]","",text)
    if re.search(r"(Contact for Correspondence(.*)Name:)", text):
        Reporter = re.search(r"(Contact for Correspondence(.*)Name:)", text).group(0)
        Reporter=Reporter.split('Name:')[1]

    elif re.search(r"(Initials/Name  First/Last  Media site Name (.*)Postal Address  Country)", Reporter):
        Reporter = re.search(r"(Initials/Name  First/Last  Media site Name (.*)Postal Address  Country)", Reporter).group(0)
        Reporter = re.sub(r"(Initials/Name  First/Last  Media site Name)|(Postal Address  Country)|\s", '', Reporter)
    else:
        Reporter=''
    return Reporter

def get_Patient_0075(text):
    if re.search(r"(Initials:(.*)Age:)", text):
        Pinitital = re.search(r"(Initials:(.*)Age:)", text)[0]
        Pinitital = re.sub(r"(Initials:|Age:|/s)",'', Pinitital)
    elif re.search(r"(Initials (.*)Age  )", text):
        Pinitital = re.search(r"(Initials (.*)Age  )", text)[0]
        Pinitital = re.sub(r"(Initials  |Age  |/s)",'', Pinitital)
    else:
        Pinitital=''
        
    if re.search(r"(Age:(.*)Age Group11:)", text):
        PAge = re.search(r"(Age:(.*)Age Group11:)", text)[0]
        PAge = re.sub(r"(Age:|Age Group11:|/s)",'', PAge)
    elif re.search(r"(Age  (.*)Date of Birth  )", text):
        PAge = re.search(r"(Age  (.*)Date of Birth  )", text)[0]
        PAge = re.sub(r"(Age  |Date of Birth  |/s)",'', PAge)
    else:
        PAge=''
    PDob = re.sub(r"[()]","",text)   
    if re.search(r"(DoB: DD-MMM-YYYY(.*)Height13:)", PDob): 
        PDob = re.search(r"(DoB: DD-MMM-YYYY(.*)Height13:)", PDob)[0]
        PDob = re.sub(r"(DoB: DD-MMM-YYYY|Height13:|/s)",'', PDob)
    elif re.search(r"(Date of Birth(.*)Ethnicity  )", text):
        PDob = re.search(r"(Date of Birth(.*)Ethnicity  )", text)[0]
        PDob = re.sub(r"(Date of Birth|Ethnicity  |/s)",'', PDob)
    else:
        PDob=''
        
    if re.search(r"(Gender:(.*)Ethnicity12:)", text):
        PGender = re.search(r"(Gender:(.*)Ethnicity12:)", text)[0]
        PGender = re.sub(r"(Gender:|Ethnicity12:|/s)",'', PGender)
    elif re.search(r"(Gender(.*)Pregnant:  )", text):
        PGender = re.search(r"(Gender(.*)Pregnant:  )", text)[0]
        PGender = re.sub(r"(Gender|Pregnant:|/s)",'', PGender)
    else:
        PGender=''
        
    return Pinitital,PAge,PDob,PGender

def get_Event_Term_0075(text):
    Narrative=''
    Narrative_new=''
    try:
        Narrative = re.sub(r"[()]","",text)
        Narrative = re.search(r"(Description of Adverse Events or Product Complaints:(.*)Other Relevant Details)", Narrative)[0]
        Narrative = re.sub(r"(Description of Adverse Events or Product Complaints:|Other Relevant Details|/s)",'', Narrative)
    except:
        Narrative=''
        pass
    try:
        
        Narrative_new = re.search(r"(seriousness etc.(.*)Complete all)", text)[0]
        Narrative_new = re.sub(r"(seriousness etc.|Complete all|/s)",'', Narrative_new)
    except:
        Narrative_new=''
        pass
    
    return Narrative+Narrative_new


def get_Narrative_0075(text):
    Narrative=''
    try:
        Narrative = re.sub(r"[()]","",text)
        Narrative = re.search(r"(Description of Adverse Events or Product Complaints:(.*)Other Relevant Details)", Narrative)[0]
        Narrative = re.sub(r"(Description of Adverse Events or Product Complaints:|Other Relevant Details|/s)",'', Narrative)
    except:
        Narrative=''
        pass
    
    return Narrative

def get_Event_Verbatim_0075(text):
    Narrative_new = ''
    try:        
        Narrative_new = re.search(r"(seriousness etc.(.*)Complete all)", text)[0]
        Narrative_new = re.sub(r"(seriousness etc.|Complete all|/s)",'', Narrative_new)
    except:
        Narrative_new=''
        pass
    
    return Narrative_new


# In[2]:


## Pfizer
def clean_text(text):
    return re.sub(r"[\n\t\u2002]", " ", text)

def get_Patient_Pfizer(pdftext):
    Pinitial = re.search(r"(PATIENT INITIALS(.*)DATE OF BIRTH)", pdftext)[0]
    Pinitial = re.sub(r"(PATIENT INITIALS|DATE OF BIRTH|/s)",'', Pinitial)
    
    PDob = re.search(r"(DATE OF BIRTH(.*)GENDER)", pdftext)[0]
    PDob = re.sub(r"(DATE OF BIRTH|GENDER|/s)",'', PDob)
    
    PGender = re.search(r"(GENDER(.*)Please email)", pdftext)[0]
    PGender = re.sub(r"(GENDER|Please email|/s)",'', PGender)

    return Pinitial,PDob,PGender

def get_ReporterVendor_Pfizer(pdftext):
    
    Reporter = re.sub(r"[().,]","",pdftext)
    Reporter = re.search(r"(NAME  Include Mr Mrs Ms Dr :(.*)FULL ADDRESS)", Reporter)[0]
    Reporter = re.sub(r"(NAME  Include Mr Mrs Ms Dr :|FULL ADDRESS|/s)",'', Reporter)
    
    Vendor = re.search(r"(VENDOR CONTACT PERSON :(.*)DATE)", pdftext)[0]
    Vendor = re.sub(r"(VENDOR CONTACT PERSON :|DATE|/s)",'', Vendor)
    
    return Reporter,Vendor

def get_Narrative_Pfizer(pdftext):
    narrative = re.search(r"(EVENT DESCRIPTION(.*)PFIZER PRODUCT NAME)", pdftext)[0]
    narrative = re.sub(r"(EVENT DESCRIPTION|PFIZER PRODUCT NAME|/s)",'', narrative)
    return narrative

#pdffile = open('Pfizer.pdf', "rb")
#pdfReader = PyPDF2.PdfFileReader(pdffile)
#pdfReader.numPages
#page = pdfReader.getPage(0)
#pdftext = page.extractText()
#pdftext = clean_text(pdftext)


# In[ ]:


def get_Reporter_form4(text):
    reporter = re.search(r"(Name/Nom:(.*)Profession:)", text)[0]
    reporter = re.sub(r"(Name/Nom:|Profession:)",'', reporter)
    
    return reporter

def get_DrugName_form4(text):
    drugname = re.search(r"(Suspect drug(.*)Special Situation)", text)[0]
    drugname = re.sub(r"[()]","",drugname)
    drugname = re.sub(r"(Suspect drug s/medicament suspects:|Adverse Events or Special Situation|/s)",'', drugname)
    return initial 


# process_form(file_name) is a general-purpose function to determine which type of form the document is and extract key fields from it


In [11]:
def get_form_number(text):
    has_form_number = re.search(r"(Form Number:\s(.*)Page:)", text)
    if has_form_number:
        form_number = has_form_number.group(0)
        form_number = re.sub(r"(Form Number:)|(Page:)|\s", '', form_number)
        if len(form_number) > 13:
            form_number = form_number[:13]
        return form_number
    return ''


def clean_text(text):
    return re.sub(r"[\n\t\u2002]", " ", text)


def process_text(file_name):
    text = docx2txt.process(file_name)
    text = re.sub(r"[\n\t\u2002]", " ", text)
    return text




'''
narrative=attachment_narrative(textlist)
narrative
'''


# Function to construct the exception message to be stored in log file
def get_exception(debug=True):
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    err_obj = {'error_code': str(exc_type),
               'error_message': str(exc_obj)
               }
    if debug:
        err_obj['error_at'] = 'EXCEPTION IN ({},LINE {} "{}")'.format(filename, lineno, line.strip())
    return err_obj
  


In [12]:
def get_attachment_contents(df_one_msg_attch):
    #df_one_msg_attch = pd.DataFrame()
    text = ''
    filenamelist = ''
    formNumber = ''
    k = 0
    attachment_count = df_one_msg_attch.shape[0]


    for i in range(attachment_count): #attachment_count
        print('*************************************************CHECK',i)
        filename = str(df_one_msg_attch['attachment_names'][i])
        #filename = 'TAKA 17 05343 ALU0068.docx'
        directory = str(df_one_msg_attch['path_of_attachement'][i])
        #directory = 'rnd-us-raw/tide/Pipeline/Exports/'
        #name = filename + 'ABCDEF'
        #name = filename + str(df_one_msg_attch['message_id'][i])    #rnd-us-raw/tide/Pipeline/Exports/
        BUCKET_NAME = 'tpc-aws-ted-dev-edpp-raw-rnd-us-east-1'  # replace with your bucket name
        KEY = directory + filename  # replace with your object key
    
        k = k + 1
        try:
            filepath = directory + filename
            print('*********************************Attachment No. ', i, ' read from S3*************************')
            # print(filename)
            ext = filename.split('.')[-1]
            print(ext)
            if k == 1:
                filenamelist = filename
            else:
                filenamelist = filenamelist + ',' + filename
    
            if ext == 'pdf':
                try:
                    pdffile = open(filepath, "rb")
                    pdfReader = PyPDF2.PdfFileReader(pdffile)
                    pagenum = pdfReader.numPages
                    pdftext = ''
                    for i in range(0, pagenum):
                        # print(i)
                        page = pdfReader.getPage(i)
                        pdftext = pdftext + page.extractText()
                    pdftext = clean_text(pdftext)
                    print('**********CHECK if text was extracted from attached .pdf file: len(pdftext): ', len(pdftext))
                    if formNumber == '':
                        formNumber = get_form_number(pdftext)
                    else:
                        formNumber = formNumber + ',' + get_form_number(pdftext)
    
                    if text == '':
                        text = pdftext
                    else:
                        text = text + ' ' + 'NewAttachment' + ' ' + pdftext
    
                except:
                    print('Here4')
                    pass
                    #PyPDF2.utils.PdfReadError
    
            elif ext == 'docx':
                try:
                    s3 = boto3.resource('s3')
                    try:
                        s3.Bucket(BUCKET_NAME).download_file(KEY, filename)
                        doctext = process_text('/databricks/driver/'+filename)
                        print('***************************************RECHING HERE1********************************')
                        #doctext = docx2txt.process()
                        #print(doctext)
                    except botocore.exceptions.ClientError as e:
                        print('Here3')
                        if e.response['Error']['Code'] == "404":
                            print("The object does not exist.")
                        else:
                            raise
    
                    # doctext = process_text(filepath)
                    if formNumber == '':
                        formNumber = get_form_number(doctext)
                    else:
                        formNumber = formNumber + ',' + get_form_number(doctext)
                    print('**********CHECK if text was extracted from attached .docx file: len(doctext): ', len(doctext))
                    if text == '':
                        text = doctext
                    else:
                        text = text + ' ' + 'NewAttachment' + ' ' + doctext
                    print('***************************************RECHING HERE2********************************')
                except zipfile.BadZipFile:
                    print('Here2')
                    pass
        except TypeError:
            print('Here1')
            pass
    print('***************************************RECHING HERE3********************************')
    print(formNumber,filenamelist,len(text))
    return formNumber, filenamelist, text

In [13]:
def get_attachment_fields(filePath, text, Attachment_Id, Relevant_Form_Flag, msg_id, batch_id, src_link_id):
    print('*******************INSIDE Attch__Extraction Function************************')
    TradeName, GenericName, VendorID, Reporter, PInitial, PAge, PDob, PGender, formNumber, Event_Term, Event_Verbatim, Narrative, status = '', '', '', '', '', '', '', '', '', '', '', '', 'success'

    global df_attch_set
    df_attch_set = pd.DataFrame()
    my_list = []
    #print('****************************CHECK*****************')
    #print(type(textli))
    #print(textli)
    
    #for i in textli:
    try:
        formNumber = get_form_number(text)
        formNumber_upper = formNumber.upper()
        print('*********************************Going for Form_Check*********************************')
        print(formNumber)
        
        # NarrativeAll=NarrativeAll+' '+formNumber
        if formNumber_upper == form_050.upper():
            print('********************G-Form-PV-050*******************')
            TradeName, GenericName = get_Drug_050(text)
            Reporter, VendorID = get_ReporterVendor_050(text)
            PInitial, PAge, PDob, PGender = get_patientdetails_050(text)
            Event_Term = get_Event_Term_050(text)
            Event_Verbatim = get_Event_Verbatim_050(text)
            Narrative = get_Narrative_050(text)
            Relevant_Form_Flag = 'Y'
            
        elif formNumber_upper == form_075.upper():
            print('********************G-Form-PV-075*******************')
            TradeName, GenericName, VendorID = get_drugNameVendor_0075(text)
            PInitial, PAge, PDob, PGender = get_Patient_0075(text)
            Reporter = get_Reporter_075(text)
            Event_Term = get_Event_Term_0075(text)
            Event_Verbatim = get_Event_Verbatim_0075(text)
            Narrative = get_Narrative_0075(text)
            Relevant_Form_Flag ='Y' 
            
        elif formNumber_upper == form_054.upper():
            print('********************G-Form-PV-054*******************')
            TradeName = get_Drug_054(text)
            Reporter = get_Reporter_054(text)
            PInitial,PAge = get_patientdetails_054(text)
            Narrative = get_Narrative_054(text)
            Event_Term = Narrative + Event_Verbatim
            Relevant_Form_Flag = 'Y'               
        elif formNumber_upper == form_3904.upper():
            print('********************FORM-0003904*******************')
            TradeName, GenericName = get_TradeGenDrug_0003904(text)
            VendorID, Reporter = get_VendorReporter_0003904(text)
            PInitial, PAge, PDob, PGender = get_Patient_0003904(text)
            Event_Term = get_Event_Term_0003904(text)
            Event_Verbatim = get_Event_Verbatim_0003904(text)
            Narrative = get_narrative_0003904(text)
            Relevant_Form_Flag = 'Y' 
            
        elif 'PFIZER' in text:
            print('********************PFIZER*******************')
            Pinitial, PDob, PGender = get_Patient_Pfizer(text)
            Reporter, Vendor = get_ReporterVendor_Pfizer(text)
            Narrative = get_Narrative_Pfizer(text)
            Event_Term = Narrative + Event_Verbatim
            Relevant_Form_Flag = 'Y'        
    except:
        print('error occured')
        status = 'exception'
        pass
        #print(formNumber)
    my_list.append([batch_id, src_link_id, filePath, formNumber, text, Attachment_Id, TradeName, GenericName, VendorID, Reporter, PInitial, PAge, PDob, PGender, Event_Term, Event_Verbatim, Narrative, msg_id, status])         
    df_attch_set = pd.DataFrame(my_list,columns=['batch_id', 'src_link_id', 'attachment_names', 'formNumber', 'attachment_text', 'attachment_id', 'TradeName', 'GenericName', 'VendorID', 'Reporter', 'PInitial', 'PAge', 'PDob', 'PGender', 'event_term', 'Event_Verbatim', 'Narrative', 'message_id', 'status'])
    print('********************Extract_Attachment DF*********************')
    print(df_attch_set.shape)
    return df_attch_set,Relevant_Form_Flag

In [14]:
def output_df(batch_id,message_id, src_link_id, attachment_id, event_term, attachment_name, formNumber, attachment_text, TradeName, GenericName, VendorID, Reporter, PInitial, PAge, PDob, PGender, Event_Verbatim, Narrative, status):
    dict_extract = {'field_name':[],'field_value':[]}
    column=['attachment_name', 'formNumber', 'attachment_text', 'TradeName', 'GenericName', 'VendorID', 'Reporter', 'PInitial', 'PAge', 'PDob', 'PGender', 'Event_Verbatim', 'Narrative', 'status']
    value=[attachment_name, formNumber, attachment_text, TradeName, GenericName, VendorID, Reporter, PInitial, PAge, PDob, PGender, Event_Verbatim, Narrative, status]
    dict_extract['batch_id']=batch_id
    dict_extract['message_id']=message_id
    dict_extract['src_link_id']=src_link_id
    dict_extract['attachment_id']=attachment_id
    dict_extract['event_term']=event_term
    dict_extract['field_name']=column
    dict_extract['field_value']=value
    return pd.DataFrame(dict_extract)

In [15]:
def extract_from_attachment(batch_id, df_email, df_attch_list, output_path):
    try:
        # df_initial['message_id','From', 'To', 'ReceivedDate', 'Subject', 'email_body', 'no_of_attachments', 'attachment_names','path_of_attachement']
        global df
        df_email['attachment'] = ''
        df_email['email_type'] = ''
        df_email['relevant_form_flag'] = ''
        my_list = []
        df_attachment = pd.DataFrame()
        df_attch_keyValPair = pd.DataFrame()
        #msg_id_list = list(df_initial['message_id'].unique())
        email_count = df_email.shape[0]
        for i in range(email_count):
            print('********LOOPING through EACH MAIL***********************')
            try:
                Relevant_Form_Flag = 'Not Applicable'
                msg_id = df_email['message_id'][i]
                attachmnt_count = df_email['no_of_attachments'][i]
                src_link_id = df_email['src_link_id'][i]
                print('*****attachmnt_count*****',type(attachmnt_count))
                print('****************RelevanceCHECK_outsideLOOP******', Relevant_Form_Flag)
                print(int(attachmnt_count))
                if int(attachmnt_count) > 0:
                    attachment = "Y"
                    df_email['attachment'][i] = attachment
                    Relevant_Form_Flag = 'N'
                    df_email['relevant_form_flag'][i] = Relevant_Form_Flag
                    print('****************RelevanceCHECK_insideLOOP******', Relevant_Form_Flag)
                    counter = 1
                    print('*********************PRINT__attachments_no. *********************', attachmnt_count)
                    df_one_msg_attch = df_attch_list[df_attch_list['message_id']==msg_id].reset_index(drop=True)
                    print(df_one_msg_attch)
                    formNumber, filename, text = get_attachment_contents(df_one_msg_attch) 
                    print('******************MAIN_DETAILS_OF EVERY_ATTACHMNT****************************************')
                    print(formNumber)
                    print(filename)
                    print('******************MAIN_DETAILS_OF EVERY_ATTACHMNT****************************************')
                    if len(text) > 20:
                        textlist = text.split('NewAttachment')
                        # print(type(textlist))
                        # textlist.pop(0)
                        print('*********SIZE OF TEXTLIST********')
                        print(len(textlist))
                    else:
                        textlist = []

                    if len(filename) > 10:
                        filenamelist = filename.split(",")
                        # filenamelist.pop(0)

                    if len(formNumber) > 10:
                        formNumberlist = formNumber.split(",")
                        # formNumberlist.pop(0)
                        print('*********SIZE OF FORMNUM_LIST********')
                        print(len(formNumberlist))

                    for count in range(attachmnt_count):
                        attch_buffer_df = pd.DataFrame()
                        if len(textlist) != 0:
                            #checklist = ['FORM-0003904', 'G-Form-PV-050', 'G-Form-PV-054', 'G-Form-PV-075', 'G-FORM-PV-0751o']
                            checklist_upper = [z.upper().strip() for z in checklist]
                            if (formNumberlist[count].strip().upper() in checklist_upper or 'PFIZER' in textlist[count]):
                                Relevant_Form_Flag = 'Y'
                                df_email['relevant_form_flag'][i] = Relevant_Form_Flag
                                Attachment_Id = str(counter) + '_Attch_for_' + msg_id
                                print('*************ATTACHMENT_LOGIC_CALLED_FOR_MAIL******', formNumberlist[count])
                                print('********************values passed:*********', filenamelist[count], len(textlist[count]),
                                      Attachment_Id, Relevant_Form_Flag, msg_id)

                                attch_buffer_df, Relevant_Form_Flag = get_attachment_fields(filenamelist[count], textlist[count],
                                                                                        Attachment_Id, Relevant_Form_Flag, msg_id,
                                                                                        batch_id,src_link_id)

                                print('**************Return DF row,column from Attch_Extraction_Function******************')
                                r, c = attch_buffer_df.shape
                                print(r, c)

                                print('*****************SAVING to ATTACHMENT_DF********')
                                df_attachment = df_attachment.append(attch_buffer_df, ignore_index=True)
                                print(df_attachment.shape)
                                for j in range(attch_buffer_df.shape[0]):
                                    extracted_df = output_df(batch_id, msg_id, src_link_id,                       
                                                             attch_buffer_df['attachment_id'][j],
                                                             attch_buffer_df['event_term'][j],
                                                             attch_buffer_df.loc[j, 'attachment_names'],
                                                             attch_buffer_df.loc[j, 'formNumber'],
                                                             attch_buffer_df.loc[j, 'attachment_text'],
                                                             attch_buffer_df.loc[j, 'TradeName'],
                                                             attch_buffer_df.loc[j, 'GenericName'],
                                                             attch_buffer_df.loc[j, 'VendorID'],
                                                             attch_buffer_df.loc[j, 'Reporter'],
                                                             attch_buffer_df.loc[j, 'PInitial'],
                                                             attch_buffer_df.loc[j, 'PAge'],
                                                             attch_buffer_df.loc[j, 'PDob'],
                                                             attch_buffer_df.loc[j, 'PGender'],
                                                             attch_buffer_df.loc[j, 'Event_Verbatim'],
                                                             attch_buffer_df.loc[j, 'Narrative'],
                                                             attch_buffer_df.loc[j, 'status'])

                                    print('*****************SAVING to TRANSPOSE_DF********')
                                    df_attch_keyValPair = df_attch_keyValPair.append(extracted_df, ignore_index=True)
                                    print(df_attch_keyValPair.shape)
                                counter = counter + 1
                else:
                    attachment = "N"
                    df_email['attachment'][i] = attachment

                print('******EMAIL_LIST_DATALIST_APPENDING****************')
                df_email['attachment'][i] = attachment
                df_email['relevant_form_flag'][i] = Relevant_Form_Flag

            except Exception as exep:
                print(get_exception())
        #df.to_csv(output_path + 'Email_Extract_id.csv')
        df_email['batch_id'] = batch_id
        df_attachment.to_csv(output_path+'Email_Attachment.csv')
        df_attch_keyValPair.to_csv(output_path+'Email_Attach_keyValPair.csv')
        df_email.to_csv(output_path+'Email_List.csv')
        return df_email,df_attachment,df_attch_keyValPair
    except Exception as exep:
        print(get_exception())
        raise exep

In [16]:
##Creating Matrix DataFrame to call ML model
def get_MessageID(df_emailattach):
    messageid_List = list(df_emailattach['message_id'].unique())
    return messageid_List


def concat_event_term_per_email(msgid, df_emailattach):
    df_emailattach_s = df_emailattach[['message_id', 'event_term']].drop_duplicates()
    df_new = df_emailattach_s[df_emailattach_s['message_id'] == msgid]
    df_final = df_new['event_term'].unique()
    eventTerm_concat = ''
    for i in range(df_final.shape[0]):
        if i == (df_final.shape[0] - 1):
            eventTerm_concat += str(df_final[i])
        else:
            eventTerm_concat += str(df_final[i]) + ','

    return eventTerm_concat


def get_eventverbiage_all_emails_in_dataframe(df_attachment):
    x = get_MessageID(df_attachment)
    df_eventverbiage_all_emails = pd.DataFrame(columns=["message_id", "event_verbiage"])
    for j in range(len(x)):
        msg_id = x[j]
        # print (msg_id)
        event_verbiage = concat_event_term_per_email(msg_id, df_attachment)
        # print(event_verbiage)
        df_eventverbiage_all_emails.loc[j, 'message_id'] = msg_id
        df_eventverbiage_all_emails.loc[j, 'event_verbiage'] = event_verbiage
    return df_eventverbiage_all_emails


def create_ml_matrix_df(batch_id,df_email_list, df_attachment, output_path):
    df_email = df_email_list[['message_id', 'Subject', 'MailBody_Text']].drop_duplicates()
    df_email['event_verbiage'] = ''
    df_ml = df_email
    check = int(df_attachment.shape[0])
    if check > 0:
        del df_email['event_verbiage']
        df_eventverbiage_all_emails = get_eventverbiage_all_emails_in_dataframe(df_attachment)
        df_ml = pd.merge(df_email, df_eventverbiage_all_emails, how='left', on='message_id')
    df_ml['batch_id']=batch_id
    df_ml.reset_index(inplace=True,drop=True)
    df_ml.to_csv(output_path + 'ML_Matrix_DataFrame.csv')    
    return df_ml

In [17]:
def create_standard_dataframe(df_input_to_process):
    # df_initial = pd.read_csv(s3_path+master_filename)
    # df_initial = pd.read_csv('s3://tpc-aws-ted-dev-edpp-raw-rnd-us-east-1/rnd-us-raw/tide/Pipeline/Data_Pipeline/export.csv')
    # count_check    
    df_initial = df_input_to_process
    r, c = df_initial.shape
    print('df_initial', r, c)
    df_email = df_initial[['batch_id','message_id', 'src_link_id', 'From', 'To', 'ReceivedDate', 'Subject', 'MailBody_Text', 'no_of_attachments']]
    df_attch_all = df_initial[['batch_id','message_id', 'src_link_id', 'no_of_attachments', 'attachment_names', 'path_of_attachement']]
    df_attch_all = df_attch_all.reset_index(drop=True)
    df_attch_list = df_attch_all[df_attch_all['no_of_attachments'] > 0]
    df_attch_list = df_attch_list.reset_index(drop=True)
    count = df_attch_list.shape[0]
    for i in range(count):
        print(df_attch_list['attachment_names'][i])
        filename = str(df_attch_list['attachment_names'][i])
        filename = filename.replace("', '", ",")
        filename = filename.replace("[", '')
        filename = filename.replace("]", '')
        filename = filename.replace("'", '')
        print(filename)
        df_attch_list['attachment_names'][i] = filename
    df_single_attch = df_attch_list[df_attch_list['no_of_attachments'] == 1]  # .reset_index(inplace=True)
    # count_check
    r, c = df_single_attch.shape
    print('df_single_attch', r, c)
    
    df_final_attch = df_single_attch
    
    df_multiple_attch = df_attch_list[df_attch_list['no_of_attachments'] > 1]  # .reset_index(inplace=True)
    # count_check
    r, c = df_multiple_attch.shape
    print('df_multiple_attch', r, c)
    df_multiple_attch = df_multiple_attch.reset_index(drop=True)
    cnt = int(df_multiple_attch.shape[0])
    if cnt > 0:
        for i in range(cnt):
            filename = df_multiple_attch['attachment_names'][i]
            filenamelist = filename.split(",")
            df_multiple_attch['attachment_names'][i] = filenamelist

        s = df_multiple_attch['attachment_names'].apply(pd.Series, 1).stack()

        s.index = s.index.droplevel(-1)
        s.name = 'attachment_names'
        del df_multiple_attch['attachment_names']
        df_multiple_attch = df_multiple_attch.join(s)
        # count_check
        r, c = df_multiple_attch.shape
        print('df_multiple_attch', r, c)
        print(df_multiple_attch['attachment_names'])
        df_final_attch = df_final_attch.append(df_multiple_attch, ignore_index=True)

    
    # count_check
    r, c = df_final_attch.shape
    print('df_final_attch', r, c)
    #print(df_final_attch['attachment_names'])
    df_email.reset_index(inplace=True,drop=True)
    #df_email['batch_id']=batch_id  
    df_email.to_csv(output_path+'df_email.csv')
    df_final_attch.reset_index(inplace=True,drop=True)
    #df_final_attach['batch_id']=batch_id
    df_final_attch.to_csv(output_path+'df_attch_list.csv')
    return df_email, df_final_attch

In [18]:
##############################------------functions used for ML MODEL--------########3

def remove_special_char(text):
  return re.sub(r"[^\x00-\x7F]", "", text)

def remove_extra_whitespace(text):
  return re.sub(r"\s+", " ", re.sub(r"[\n\t\r]", " ", text))

def remove_email_phone(text):
    #remove email IDs
    email_pattern = re.compile(r"([a-z0-9._-]+)@[a-z0-9.-]+\.[a-z]{2,3}", flags = re.IGNORECASE)
    text = re.sub(email_pattern, " ", text) 
    
    #remove Canadian phone numbers
    phone_pattern1 = re.compile(r"(\+\d[\s-]*)*\d{10}")
    phone_pattern2 = re.compile(r"(\+\d[\s-]*)*\(*\d{3}\)*[\.\s-]*\d{3}[\.\s-]*\d{4}")
    text = re.sub(phone_pattern1, ' ', text)
    text = re.sub(phone_pattern2, ' ', text)
    
    return text

def mask_with_regex(text):
    #mask email IDs
    email_pattern = re.compile(r"([a-z0-9._-]+)@[a-z0-9.-]+\.[a-z]{2,3}", flags = re.IGNORECASE)
    text = re.sub(email_pattern, "<<XXX.XXX@XXX.XXX>>", text) 
    
    #mask Canadian phone numbers
    phone_pattern1 = re.compile(r"(\+\d[\s-]*)*\d{10}")
    phone_pattern2 = re.compile(r"(\+\d[\s-]*)*\(*\d{3}\)*[\.\s-]*\d{3}[\.\s-]*\d{4}")
    text = re.sub(phone_pattern1, '<<(XXX)-XXX-XXXX>>', text)
    text = re.sub(phone_pattern2, '<<(XXX)-XXX-XXXX>>', text)
    
    return text

def clean_mail(text, WARNING_MESSAGES):
    #remove non-ascii characters
    text = remove_special_char(text) 
    
    #remove extra whitespace
    text = remove_extra_whitespace(text) 

    #remove warning messages
    for warning in WARNING_MESSAGES:
        text = re.sub(warning, " ", text)

    #remove sender, time sent, and recipient
    text = re.sub(r"((from:)|(sent:))(.)+subject", " Subject", text, flags = re.IGNORECASE)
    
    #remove urls and email ids (urls and email ids are usually in <>)
    text = re.sub(r"<(.)*?>", " ", text, flags = re.IGNORECASE)
    text = re.sub(r"\[mailto:(.)*?\]", " ", text)
    text = re.sub(r"www.(.)+?.com", '', text)   

    #remove email IDs and phone numbers
    text = remove_email_phone(text)
    
    #remove Canadian zip codes
    text = re.sub(r"[a-z][0-9][a-z] [0-9][a-z][0-9]", " ", text, flags = re.IGNORECASE)
    
    #remove images
    re.sub(r"cid:image.(.)*\s", " ", text)
    
    #remove dashes and repeated underscores
    text = re.sub(r"-+", " ", text)
    text = re.sub(r"__+", " ", text)
    
    #remove email signatures
    text = re.sub(r"((regards)|(thanks,)|(best,))(.)+subject", " ", text, flags = re.IGNORECASE)
    text = re.sub(r"((regards)|(thanks,)|(best,))(.*)", " ", text, flags = re.IGNORECASE)
    
    #remove any extra whitespace that may have been created
    text = re.sub(r"\s+", " ", text)
    
    return text
#functions for recall precision and F1 score
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_model_token_DNN_AE(bucketname,key,modelkey):
  s3client = boto3.client('s3')
  response = s3client.get_object(Bucket=bucketname, Key=key)
  tokenizer_load = response['Body'].read()
  tokenizer_load = pickle.loads(tokenizer_load)

  result = s3client.download_file(bucketname,modelkey, "model_23042020.h5")
  model = load_model("model_23042020.h5",custom_objects={'f1_m':f1_m,'recall_m':recall_m,'precision_m':precision_m})
  
  return tokenizer_load,model

In [19]:
def remove_stopwords(text, additional_stopwords):
    nltk_stop_words = set(stopwords.words('english')) 
    
    stop_words = nltk_stop_words | additional_stopwords
    
    word_tokens = word_tokenize(text)     
    result = []
    for token in word_tokens:
      token_lower = token.lower()
      if token_lower not in stop_words: 
        result.append(token)
        
    return(" ".join(result))
  
def prepare_data(df, mail_col, narrative_col, warning_msg_file, custom_stopwords_file):
  #load warning messages and custom stopwords
  warnings = pd.read_csv(warning_msg_file)
  WARNING_MESSAGES = list(warnings.Message)
  
  stopwords_df = pd.read_csv(custom_stopwords_file)
  CUSTOM_STOPWORDS = set(stopwords_df.word)
  
  #clean data
  df.fillna('',inplace=True)
  df[mail_col] = df[mail_col].apply(lambda x: clean_mail(str(x), WARNING_MESSAGES))
  df['text']= df[mail_col]+df[narrative_col]
  df.fillna('0',inplace=True)
  df['text'] = df['text'].apply(lambda x:remove_stopwords(x, CUSTOM_STOPWORDS))
  return(df)

In [20]:
def get_AE_Model_output(df_blind):
  df_blind.fillna('',inplace=True)  
  df_blind.reset_index(inplace=True)
  df_blind.drop(columns='index',axis=0,inplace=True)
  ## Clean Mail body
  #df_blind['MailBody_Text'] = df_blind['MailBody_Text'].apply(lambda x: clean_mail(str(x)))
  #df_blind['text']=df_blind['MailBody_Text']+df_blind['event_verbiage']
  
  df_blind = prepare_data(df_blind, 'MailBody_Text', 'event_verbiage', warning_msg_file, custom_stopwords_file)
   
  tokenizer,model=get_model_token_DNN_AE(bucketname,Tokenizerpath,ModelPath)
  
  blind_mail= df_blind['text']
  blind_mail_x = tokenizer.texts_to_matrix(blind_mail)
  
  
  valid_result=pd.DataFrame(columns=['message_id', 'Subject', 'MailBody_Text', 'event_verbiage','Prob_Other_Email','Prob_AE_Email','Prob_Global_EMail','predicted_label'])
  
  text_labels=np.array([0,1,2])

  for i in range(len(blind_mail_x)):
      #print(i)
      predictions=model.predict(np.array([blind_mail_x[i]]))
      predicted_label = text_labels[np.argmax(predictions)]
      valid_result=valid_result.append({'message_id':df_blind.loc[i,'message_id'],
                                        'Subject':df_blind.loc[i,'Subject'],
                                        'MailBody_Text':df_blind.loc[i,'MailBody_Text'],
                                        'event_verbiage':df_blind.loc[i,'event_verbiage'],
                                        'Prob_Other_Email':predictions[0][0],
                                        'Prob_AE_Email':predictions[0][1],
                                        'Prob_Global_EMail':predictions[0][2],
                                        'predicted_label':predicted_label},ignore_index=True) 
  
  return valid_result


def get_fatal_Model_output(df_blind,bucketname,Tokenizerpath,ModelPath):
  df_blind.fillna('',inplace=True)
  
  df_blind.reset_index(inplace=True)
  df_blind.drop(columns='index',axis=0,inplace=True)
  ## Clean Mail body
  #df_blind['MailBody_Text'] = df_blind['MailBody_Text'].apply(lambda x: clean_mail(str(x)))
  #df_blind['text']=df_blind['MailBody_Text']+df_blind['event_verbiage']
  
  df_blind = prepare_data(df_blind, 'MailBody_Text', 'event_verbiage', warning_msg_file, custom_stopwords_file)
   
  tokenizer,model=get_model_token_DNN_AE(bucketname,Tokenizerpath,ModelPath)
  
  blind_mail= df_blind['text']
  blind_mail_x = tokenizer.texts_to_matrix(blind_mail)
  
  
  valid_result=pd.DataFrame(columns=['message_id', 'Subject', 'MailBody_Text', 'event_verbiage','Prob_Fatal','Prob_Non-Fatal'])
  
  text_labels=np.array([0,1,2])

  for i in range(len(blind_mail_x)):
      #print(i)
      predictions=model.predict(np.array([blind_mail_x[i]]))
      predicted_label = text_labels[np.argmax(predictions)]
      valid_result=valid_result.append({'message_id':df_blind.loc[i,'message_id'],
                                        'Subject':df_blind.loc[i,'Subject'],
                                        'MailBody_Text':df_blind.loc[i,'MailBody_Text'],
                                        'event_verbiage':df_blind.loc[i,'event_verbiage'],
                                        'Prob_Fatal':predictions[0][0],
                                        'Prob_Non-Fatal':predictions[0][1]},
                                        ignore_index=True)   
  
  return valid_result


def get_serious_Model_output(df_blind,bucketname,Tokenizerpath,ModelPath):
  df_blind.fillna('',inplace=True)
  
  df_blind.reset_index(inplace=True)
  df_blind.drop(columns='index',axis=0,inplace=True)
  ## Clean Mail body
  #df_blind['MailBody_Text'] = df_blind['MailBody_Text'].apply(lambda x: clean_mail(str(x)))
  #df_blind['text']=df_blind['MailBody_Text']+df_blind['event_verbiage']
  
  df_blind = prepare_data(df_blind, 'MailBody_Text', 'event_verbiage', warning_msg_file, custom_stopwords_file)
   
  tokenizer,model=get_model_token_DNN_AE(bucketname,Tokenizerpath,ModelPath)
  
  blind_mail= df_blind['text']
  blind_mail_x = tokenizer.texts_to_matrix(blind_mail)
  
  
  valid_result=pd.DataFrame(columns=['message_id', 'Subject', 'MailBody_Text', 'event_verbiage','Prob_Serious','Prob_Non-Serious'])
  
  text_labels=np.array([0,1,2])

  for i in range(len(blind_mail_x)):
      #print(i)
      predictions=model.predict(np.array([blind_mail_x[i]]))
      predicted_label = text_labels[np.argmax(predictions)]
      valid_result=valid_result.append({'message_id':df_blind.loc[i,'message_id'],
                                        'Subject':df_blind.loc[i,'Subject'],
                                        'MailBody_Text':df_blind.loc[i,'MailBody_Text'],
                                        'event_verbiage':df_blind.loc[i,'event_verbiage'],
                                        'Prob_Serious':predictions[0][0],
                                        'Prob_Non-Serious':predictions[0][1]},
                                        ignore_index=True)   
  
  return valid_result

In [21]:
#************************************** SpecialCase logic********************************************
#Create a TF matrix given a corpus
def tf_Matrix(x,y):
    corpus = x.append(y)
    vectorizer = CountVectorizer()
    matrix=vectorizer.fit_transform(corpus)
    return matrix
  

def get_exact_match(text,vocab_special):
  for word in vocab_special:
      if word in text:
        word_found=word
        break;
      else:
        word_found='No'
  return word_found

def cosine_similar(x, y, threshold,vocab_special):
    df_Other = x[x.predicted_label!=1]
    x=x[x.predicted_label==1]
    x.reset_index(inplace=True)
    x.drop(columns='index',inplace=True)
    #create the term frequency matrix
    x['text']=x['MailBody_Text']+x['event_verbiage']
    matrix = tf_Matrix(x.text, y.text)
    len_x = len(x)
    len_y = len(y)

    #get cosine similarity matrix
    C = cosine_similarity(matrix[:len_x],matrix[len_x:len_x+len_y]) 

    #get the maximum cosine value of each row (i.e. each document in x)
    x.loc[:,'cosine_similarity'] = C.max(axis=1) 


    x['Special'] = x['cosine_similarity'].apply(lambda x: 'Y' if x > threshold else 'N')
    #### Exact Match
    x_temp=x[x.Special=='N']
    x_temp['word'] = x_temp['text'].apply(lambda x:get_exact_match(x,vocab_special))
    x_temp.reset_index(inplace=True)
    x_temp.drop(columns='index',inplace=True)
    for i in range(0,x_temp.shape[0]):
      if (x_temp.loc[i,'word']=='No'):
        x_temp.loc[i,'Special']='N'
      else:
        x_temp.loc[i,'Special']='Y'
    print(x_temp.word.value_counts())
    ## Start Merge
    x.drop(columns=['predicted_label','text','cosine_similarity'],inplace=True)
    x_temp.drop(columns=['predicted_label','text','cosine_similarity','word'],inplace=True)
    df_Other.drop(columns='predicted_label',inplace=True)
    df_Other['Special']='NA'

    match_result = x[x.Special=='Y']
    match_result=match_result.append(x_temp)
    match_result=match_result.append(df_Other)

    return match_result

In [22]:
######################------------ MAIN FUNCTION To Pass & Get Values from ML MODEL--------##############
def get_Model_output(df_blind):
  print('*****************************ML MODEL CALLED SUCCESSFULLY********************************')
  valid_result1 = get_AE_Model_output(df_blind)
  valid_result2 =  get_fatal_Model_output(df_blind,bucketname,Tokenizerpath_fatal,ModelPath_fatal)
  valid_result2 = valid_result2[list(set(valid_result2.columns)-set(valid_result1.columns))]
  valid_result3 = get_serious_Model_output(df_blind,bucketname,Tokenizerpath_serious,ModelPath_serious)
  valid_result3 = valid_result3[list(set(valid_result3.columns)-set(valid_result1.columns))]
  out = pd.concat([valid_result1,valid_result2,valid_result3],axis=1)
  y = pd.DataFrame(vocab_special,columns=['text'])
  similarity_df=cosine_similar(out,y,threshold,vocab_special)
  similarity_df.reset_index(inplace=True,drop=True)
  similarity_df.to_csv(output_path+'ml_output.csv')
  return(similarity_df)


In [23]:
############------FUNCTIONS TO DETERMIINE EMAIL TYPES using both df_attachment & Email_List--------##############
def checkLocal(email_body,sent_from):
    local_id_pattern = re.compile(local_pattern, flags=re.IGNORECASE)
    lid_pattern = local_id_pattern
    sender = local_id_sender
    local_ids = re.findall(lid_pattern,email_body)
    local_ids = list(dict.fromkeys(local_ids))
    if local_ids and sent_from==sender:
        return True
    else:
        return False
    
def checkGlobal(email_body):
    
    gid_pattern = global_id_pattern
    global_ids = re.findall(gid_pattern,email_body)
    global_ids = list(dict.fromkeys(global_ids))
    if global_ids:
        return True
    else:
        return False
    
def get_MessageID(df_emailattach):
    df_emailattach_s = df_emailattach['message_id'].unique()
    messageid_List = []
    for i in range(df_emailattach_s.shape[0]):
        messageid_List.append(df_emailattach_s[i])
        
    return messageid_List


def get_Global_DF(df_ip,msgid):
    
    #df_emailattach=pd.read_csv(output_path+'Email_List_New.csv')
    local_id_pattern = re.compile(local_pattern, flags=re.IGNORECASE)
    df_rel = df_ip[df_ip['message_id']==msgid]
    df_rel.drop(["Local_Case_Ids", "Global_Case_Ids"], axis = 1, inplace = True)
    df_mb_final = df_rel['MailBody_Text'].unique()
    email_body = df_mb_final[0]
    
    global_ids = re.findall(global_id_pattern,email_body)
    unique_global_ids = list(dict.fromkeys(global_ids))
    glb_exp_count = len(unique_global_ids)
    glb_real_count = 0
    df_buffer = pd.DataFrame()
    df_final = pd.DataFrame()
    lines = [line for line in email_body.split('\n') if line.strip('') != '']
    local_global, local_global_pair = [], {}
    local_id_get, global_id_get = False, False
    if len(unique_global_ids)==1 :
      global_id = unique_global_ids[0]
      local_ids = re.findall(local_id_pattern, email_body)
      if (len(local_ids)==0):
          local_id = 'NA'
      else:
          local_id = local_ids[0]                
      local_global.append(local_id)
      local_global.append(global_id)
      local_global_pair[local_global[0]] = local_global[1]
    else :
      local_ids = re.findall(local_id_pattern, email_body)
      global_ids = re.findall(global_id_pattern, email_body)
      localid_cnt = len(local_ids)
      globalid_cnt = len(global_ids)
      if (localid_cnt == globalid_cnt):
        for i in range(len(global_ids)):
          for j in range(len(local_ids)):
            if (i == j):
              local_global.append(local_ids[j])
              local_global.append(global_ids[i])
              local_global_pair[local_global[0]] = local_global[1]
              local_global = []
      
            
    df1 = pd.DataFrame()
    print(local_global_pair)
    global_msg_pair={}
    if local_global_pair:
      global_msg_pair[msgid] = local_global_pair
      print(local_global_pair)
      print(global_msg_pair)
      df1.loc[0,'message_id'] = msgid
      df1.loc[0,'Local_Case_Ids'] = None
      df1['Global_Case_Ids']=df1['message_id'].map(global_msg_pair)
      print(df1)
      df_final = pd.merge(df_rel,df1,how = 'inner', on=['message_id'])
      
    df_buffer = df_buffer.append(df_final)
    return df_buffer


def get_Local_DF(df_ip,msgid):
    local_id_pattern = re.compile(local_pattern, flags=re.IGNORECASE)
    df_rel = df_ip[df_ip['message_id']==msgid]
    df_rel.drop(["Local_Case_Ids", "Global_Case_Ids"], axis = 1, inplace = True)
    df_mb_final = df_rel['MailBody_Text'].unique()
    email_body = df_mb_final[0]
    
    df_buffer = pd.DataFrame()
    df_final = pd.DataFrame()
    lines = [line for line in email_body.split('\n') if line.strip('') != '']
    local_global, local_global_pair = [], {}
    for line in lines:
        local_ids = re.findall(local_id_pattern, line)
        if local_ids:
            local_global.append(local_ids[0])
            
        df1 = pd.DataFrame()
        if local_global:
            #print(local_global)
            df1.loc[0,'message_id'] = msgid
            df1.loc[0,'Local_Case_Ids'] = local_global
            df1['Global_Case_Ids']=None
            #print(df1)
            df_final = pd.merge(df_rel,df1,how = 'inner', on=['message_id'])
            
    df_buffer = df_buffer.append(df_final)
    return df_buffer
    #return df1


def get_Mail_Type_details(msgid,AEConfidenceScore, OtherConfidenceScore, GlobalEmailConfidenceScore,FatalConfidenceScore,NonFatalConfidenceScore,SeriousConfidenceScore,NonSeriousConfidenceScore,Special_AE_Form,df_emailattach):
    df_emaillist = pd.DataFrame()
    print(msgid)
    ### Check Mail Type
    print('************************VALUE_CHECKS*********************************************')
    print(AEConfidenceScore,AEThreshold_ConfScore)
    print('AEConfidenceScore', type(AEConfidenceScore))
    print('AEThreshold_ConfScore', type(AEThreshold_ConfScore))
    print(OtherConfidenceScore,OtherThreshold_ConfScore)
    print('OtherConfidenceScore', type(OtherConfidenceScore))
    print('OtherThreshold_ConfScore', type(OtherThreshold_ConfScore))
    print(GlobalEmailConfidenceScore,GlobalThreshold_ConfScore)
    print('GlobalEmailConfidenceScore', type(GlobalEmailConfidenceScore))
    print('GlobalThreshold_ConfScore', type(GlobalThreshold_ConfScore))
    
    if (AEConfidenceScore > AEThreshold_ConfScore):
        Mail_Type = 'NEW'
    elif (OtherConfidenceScore > OtherThreshold_ConfScore):
        Mail_Type ='OTHER'
    elif (GlobalEmailConfidenceScore > GlobalThreshold_ConfScore):
        Mail_Type = 'GLOBAL CASE SENT'
    else:
        Mail_Type = 'UNCLASSIFIED'
    #Mail_Type = 'UNCLASSIFIED'
        
    ### Additional check for Other and Unclassified Emails
    df_rel1 = df_emailattach[df_emailattach['message_id']==msgid]
    df_rel = df_rel1.reset_index()
    ## get Mail Body
    df_mb_final = df_rel['MailBody_Text'].unique()
    #df_mb_final = df_mb_final1.reset_index()
    Mail_Body = df_mb_final[0]
    ## get Sender
    df_frm_final = df_rel['From'].unique()
    #df_frm_final = df_frm_final1.reset_index()
    Sent_From = df_frm_final[0]
    df1 = df_emailattach[df_emailattach['message_id']==msgid]
    df1['Global_Case_Ids'] = ''
    df1['Local_Case_Ids'] = ''
    df1['AEConfidenceScore'] = AEConfidenceScore
    df1['OtherConfidenceScore'] = OtherConfidenceScore
    df1['GlobalEmailConfidenceScore'] = GlobalEmailConfidenceScore
    df1['FatalConfidenceScore'] = FatalConfidenceScore 
    df1['NonFatalConfidenceScore'] = NonFatalConfidenceScore 
    df1['SeriousConfidenceScore'] = SeriousConfidenceScore
    df1['NonSeriousConfidenceScore'] = NonSeriousConfidenceScore
    df1['Special_AE_Form'] = Special_AE_Form 
    Mail_Type_Final = Mail_Type
    print(Mail_Type_Final)
    if (Mail_Type == 'UNCLASSIFIED' or Mail_Type == 'OTHER'):
        print('****** Inside Global/Local Check *******')
        isGlobal = checkGlobal (Mail_Body)
        if(isGlobal == True):
            print('****** Inside Global Check *******')
            Mail_Type_Final = 'GLOBAL CASE SENT'
            df1['email_type'] = Mail_Type_Final
        else :
            isLocal = checkLocal (Mail_Body,Sent_From)
            if (isLocal == True):
                print('****** Inside Local Check *******')
                Mail_Type_Final = 'LOCAL CASE SENT'
                df1['email_type'] = Mail_Type_Final
            else :
                print('****** Inside Unclassified/Other Check *******')
                Mail_Type_Final = Mail_Type
                df1['email_type'] = Mail_Type_Final
    else:
        print('****** Inside AE/Global Check *******')
        Mail_Type_Final = Mail_Type
        df1['email_type'] = Mail_Type_Final
        
    print(df1)
    
    if (Mail_Type_Final == 'GLOBAL CASE SENT'):
        df_gl_final = get_Global_DF(df1,msgid)
        print('***** HERE ****')
        print(df_gl_final.columns)
        #df_gl_final = df_gl_final[df_gl_final['Global_Case_Ids'] != None]
        df_emaillist = df_emaillist.append(df_gl_final)
    elif (Mail_Type_Final == 'LOCAL CASE SENT'):
        df_ll_final = get_Local_DF(df1,msgid)
        df_emaillist = df_emaillist.append(df_ll_final)
        print('Inside Local')
    else:
        df_emaillist = df_emaillist.append(df1)
      
    return df_emaillist
  

def get_mail_types(df_ml_output,df_email_list):
    m = df_ml_output.shape[0]
    df_final_output = pd.DataFrame()
    for i in range(m):
        print('******value of i ********* ',i)
        msgid = df_ml_output['message_id'][i]
        print(msgid)
        AEConfidenceScore = df_ml_output['Prob_AE_Email'][i]
        OtherConfidenceScore = df_ml_output['Prob_Other_Email'][i]
        GlobalEmailConfidenceScore = df_ml_output['Prob_Global_EMail'][i]
        FatalConfidenceScore = df_ml_output['Prob_Fatal'][i]
        NonFatalConfidenceScore = df_ml_output['Prob_Non-Fatal'][i]
        SeriousConfidenceScore = df_ml_output['Prob_Serious'][i]
        NonSeriousConfidenceScore = df_ml_output['Prob_Non-Serious'][i]
        Special_AE_Form = df_ml_output['Special'][i]
        df_op = get_Mail_Type_details(msgid,AEConfidenceScore,OtherConfidenceScore,GlobalEmailConfidenceScore,FatalConfidenceScore,NonFatalConfidenceScore,SeriousConfidenceScore,NonSeriousConfidenceScore,Special_AE_Form,df_email_list)
        df_final_output = df_final_output.append(df_op,ignore_index=True)
    df_final_output.to_csv(output_path+'Email_Type_Final.csv',index=False)
    return df_final_output
  


In [24]:
def get_localid():
    df = pd.read_csv(output_path+'Local_ID.csv')
    lid = df['Local_ID'][0]
    return int(lid)

def set_localid(lid):
    df = pd.DataFrame()
    df.loc[0, 'Local_ID'] = lid
    df.to_csv(output_path+'Local_ID.csv',index=False)

In [25]:
set_localid(100000)

In [26]:
## Code to read incremental data in final Flattened DataFrame
def get_flatten_df(local_id):
    df_combined = pd.DataFrame()
    df_flatten_temp = pd.read_csv(output_path+'Email_Flattened_Temp.csv')
    df_flatten_temp.drop(["Audit_Flag"], axis = 1, inplace = True)
    df_flatten = pd.read_csv(output_path+'Email_Flattened_Final.csv')
    df_combined = df_combined.append(df_flatten_temp)
    df_combined = df_combined.append(df_flatten)
    df_fil = df_combined[df_combined['Local_Case_Ids_New'] == local_id].drop_duplicates()
    return df_fil

def get_df_flatten():
    df_combined = pd.DataFrame()
    df_flatten_temp = pd.read_csv(output_path+'Email_Flattened_Temp.csv')
    df_flatten_temp.drop(["Audit_Flag"], axis = 1, inplace = True)
    df_flatten = pd.read_csv(output_path+'Email_Flattened_Final.csv')
    df_combined = df_combined.append(df_flatten_temp)
    df_combined = df_combined.append(df_flatten)
    df_combined.drop_duplicates()
    return df_combined

In [27]:
## Code to write incremental data in final Flattened DataFrame
def write_email_flattened_df(df_flatten_global):
    df_flatten_final = pd.read_csv(output_path + 'Email_Flattened_Final.csv')
    
    df_final = pd.DataFrame()
    df_int = pd.DataFrame()
    
    df_flatten_global_ins = df_flatten_global[df_flatten_global["Audit_Flag"] == 'I']
    df_flatten_global_ins.drop(["Audit_Flag"], axis = 1, inplace = True)
    df_final = df_final.append(df_flatten_global_ins)
    
    df_flatten_global_upd = df_flatten_global[df_flatten_global["Audit_Flag"] == 'U']
    df_flatten_global_upd.drop(["Audit_Flag"], axis = 1, inplace = True)
    
    localid_list_batch_ins = df_flatten_global_ins['Local_Case_Ids_New'].unique()
    localid_list_batch = df_flatten_global_upd['Local_Case_Ids_New'].unique()
    localid_list_hist = df_flatten_final['Local_Case_Ids_New'].unique()
    
    if (len(localid_list_hist) == 0) :
        df_int = df_int.append(df_flatten_global_upd)
    else :
        for i in range(len(localid_list_hist)):
            if (localid_list_hist[i] in localid_list_batch):
                df1 = df_flatten_global_upd[df_flatten_global_upd['Local_Case_Ids_New'] == localid_list_hist[i]]
                df_int = df_int.append(df1)
            else :
                df2 = df_flatten_final[df_flatten_final['Local_Case_Ids_New'] == localid_list_hist[i]]
                df_int = df_int.append(df2)
    df_final = df_final.append(df_int)
    df_final.to_csv(output_path + 'Email_Flattened_Final.csv', index=False)
    return df_final

In [28]:
def new_email_flatten(df3_new_email):
    df_email_new = pd.DataFrame()
    df3_new = df3_new_email.reset_index(drop=True)
    for i in range(df3_new.shape[0]):
        localid = get_localid()
        localid += 1
        local_id = 'CA-' + str(localid)
        msgid = df3_new['message_id'][i]
        src_link_id = df3_new['src_link_id'][i]
        #src_link_id = int(src_link_id1)
        sent_from = df3_new['From'][i]
        subject = df3_new['Subject'][i]
        date = df3_new['ReceivedDate'][i]
        ae_confidence_score = df3_new['AEConfidenceScore'][i]
        other_confidence_score = df3_new['OtherConfidenceScore'][i]
        global_confidence_score = df3_new['GlobalEmailConfidenceScore'][i]
        df_email_new.loc[i,'Global_Case_Ids'] = None
        df_email_new.loc[i,'Local_Case_Ids_New'] = local_id
        df_email_new.loc[i,'Local_Case_Ids_New_DateTime'] = date
        df_email_new.loc[i,'Local_Case_Ids_Local'] = None
        df_email_new.loc[i,'Local_Case_Ids_Local_DateTime'] = None
        df_email_new.loc[i,'Local_Case_Ids_Global'] = None
        df_email_new.loc[i,'Local_Case_Ids_Global_DateTime'] = None
        df_email_new.loc[i,'message_ID'] = msgid
        df_email_new.loc[i,'src_link_id'] = src_link_id
        df_email_new.loc[i,'From'] = sent_from
        df_email_new.loc[i,'Subject'] = subject
        df_email_new.loc[i,'Email_Date_Time'] = date
        df_email_new.loc[i,'AEConfidenceScore'] = ae_confidence_score
        df_email_new.loc[i,'OtherConfidenceScore'] = other_confidence_score
        df_email_new.loc[i,'GlobalEmailConfidenceScore'] = global_confidence_score
        df_email_new.loc[i,'Audit_Flag'] = 'I'
        set_localid(localid)
    
    df_email_new.to_csv(output_path+'Email_Flattened_Temp.csv',index=False)
    return df_email_new

In [29]:
def local_email_df(df3_local_email):
    
    df_email_local = pd.DataFrame()
    df3_local = df3_local_email.reset_index(drop=True)
    df3_local = df3_local.sort_values(by ='Local_Case_Ids', na_position='last')
    localid_list = df3_local['Local_Case_Ids'].unique()
    date_list = df3_local['ReceivedDate'].unique()
    
    for i in range(len(localid_list)):
        date = date_list[i]
        if (',' in localid_list[i]):
                llist = localid_list[i].split(sep=',')
                for j in range(len(llist)):
                    localid = llist[j]
                    df_flatten_email = get_flatten_df(localid)
                    df_flatten = df_flatten_email.reset_index(drop=True)
                    if (df_flatten.shape[0] > 0):
                        local_case_id_local = df_flatten['Local_Case_Ids_New'].unique()
                        df_flatten = df_flatten.reset_index()
                        df_flatten.loc[0,'Local_Case_Ids_Local'] = local_case_id_local
                        df_flatten.loc[0,'Local_Case_Ids_Local_DateTime'] = date
                        df_flatten.loc[0,'Audit_Flag'] = 'U'
                        df_email_local = df_email_local.append(df_flatten)
                    else :
                        df3_local_rel2 = df3_local[df3_local['Local_Case_Ids'] == localid_list[i]]
                        df3_local_rel = df3_local_rel2.reset_index(drop=True)
                        msgid = df3_local_rel['message_id'][0]
                        src_link_id = df3_local_rel['src_link_id'][0]
                        sent_from = df3_local_rel['From'][0]
                        subject = df3_local_rel['Subject'][0]
                        date = df3_local_rel['ReceivedDate'][0]
                        ae_confidence_score = df3_local_rel['AEConfidenceScore'][0]
                        other_confidence_score = df3_local_rel['OtherConfidenceScore'][0]
                        global_confidence_score = df3_local_rel['GlobalEmailConfidenceScore'][0]
                        df_i = pd.DataFrame()
                        df_i.loc[0,'Global_Case_Ids'] = None
                        df_i.loc[0,'Local_Case_Ids_New'] = None
                        df_i.loc[0,'Local_Case_Ids_New_DateTime'] = None
                        df_i.loc[0,'Local_Case_Ids_Local'] = localid
                        df_i.loc[0,'Local_Case_Ids_Local_DateTime'] = date
                        df_i.loc[0,'Local_Case_Ids_Global'] = None
                        df_i.loc[0,'Local_Case_Ids_Global_DateTime'] = None
                        df_i.loc[0,'message_ID'] = msgid
                        df_i.loc[0,'src_link_id'] = src_link_id
                        df_i.loc[0,'From'] = sent_from
                        df_i.loc[0,'Subject'] = subject
                        df_i.loc[0,'Email_Date_Time'] = date
                        df_i.loc[0,'AEConfidenceScore'] = ae_confidence_score
                        df_i.loc[0,'OtherConfidenceScore'] = other_confidence_score
                        df_i.loc[0,'GlobalEmailConfidenceScore'] = global_confidence_score
                        df_i.loc[0,'Audit_Flag'] = 'I'
                        df_email_local = df_email_local.append(df_i)
        else :
                localid = localid_list[i]
                df_flatten_email = get_flatten_df(localid)
                df_flatten = df_flatten_email.reset_index(drop=True)
                if (df_flatten.shape[0] > 0):
                    local_case_id_local = df_flatten['Local_Case_Ids_New'].unique()
                    df_flatten = df_flatten.reset_index()
                    df_flatten.loc[0,'Local_Case_Ids_Local'] = local_case_id_local
                    df_flatten.loc[0,'Local_Case_Ids_Local_DateTime'] = date
                    df_flatten.loc[0,'Audit_Flag'] = 'U'
                    df_email_local = df_email_local.append(df_flatten)
                else :
                    df3_local_rel2 = df3_local[df3_local['Local_Case_Ids'] == localid]
                    df3_local_rel = df3_local_rel2.reset_index(drop=True)
                    msgid = df3_local_rel['message_ID'][0]
                    src_link_id = df3_local_rel['src_link_id'][0]
                    sent_from = df3_local_rel['From'][0]
                    subject = df3_local_rel['Subject'][0]
                    #date = df3_local_rel['Date']
                    ae_confidence_score = df3_local_rel['AEConfidenceScore'][0]
                    other_confidence_score = df3_local_rel['OtherConfidenceScore'][0]
                    global_confidence_score = df3_local_rel['GlobalEmailConfidenceScore'][0]
                    df_i = pd.DataFrame()
                    df_i.loc[0,'Global_Case_Ids'] = None
                    df_i.loc[0,'Local_Case_Ids_New'] = None
                    df_i.loc[0,'Local_Case_Ids_New_DateTime'] = None
                    df_i.loc[0,'Local_Case_Ids_Local'] = localid
                    df_i.loc[0,'Local_Case_Ids_Local_DateTime'] = date
                    df_i.loc[0,'Local_Case_Ids_Global'] = None
                    df_i.loc[0,'Local_Case_Ids_Global_DateTime'] = None
                    df_i.loc[0,'message_ID'] = msgid
                    df_i.loc[0,'src_link_id'] = src_link_id
                    df_i.loc[0,'From'] = sent_from
                    df_i.loc[0,'Subject'] = subject
                    df_i.loc[0,'Email_Date_Time'] = date
                    df_i.loc[0,'AEConfidenceScore'] = ae_confidence_score
                    df_i.loc[0,'OtherConfidenceScore'] = other_confidence_score
                    df_i.loc[0,'GlobalEmailConfidenceScore'] = global_confidence_score
                    df_i.loc[0,'Audit_Flag'] = 'I'
                    df_email_local = df_email_local.append(df_i)
    #print(df_email_local)                    
    return df_email_local

In [30]:
def global_email_df(df3_global_email):
    df3_global = df3_global_email.reset_index(drop=True)
    df3_global = df3_global.sort_values(by ='Local_Case_Ids', na_position='last')
    date_list = df3_global['ReceivedDate'].unique()
    globalid_list = df3_global['Global_Case_Ids'].unique()
    df_email_global = pd.DataFrame()
    globalid_list
    
    for i in range(len(globalid_list)):
        date = date_list[i]
        globalid_l = globalid_list[i].replace('{','').replace('}','').replace('\'','').split(sep=',')
        globalid_l.sort()
        for j in range(len(globalid_l)):
            globalid = globalid_l[j].split(sep=':')[1]
            localid = globalid_l[j].split(sep=':')[0]
            df_flatten_email = get_flatten_df(localid)
            df_flatten = df_flatten_email.reset_index(drop=True)
            if (df_flatten.shape[0] > 0):
                local_case_id_local = df_flatten['Local_Case_Ids_Local'].unique()
                df_flatten = df_flatten.reset_index()
                df_flatten.loc[0,'Local_Case_Ids_Global'] = local_case_id_local
                df_flatten.loc[0,'Local_Case_Ids_Global_DateTime'] = date
                df_flatten.loc[0,'Global_Case_Ids'] = globalid
                df_flatten.loc[0,'Audit_Flag'] = 'U'
                df_email_global = df_email_global.append(df_flatten)
            else :
                #df_global_rel2 = pd.DataFrame()
                df3_global_rel2 = df3_global[df3_global['Global_Case_Ids'] == globalid_list[i]]
                df3_global_rel = df3_global_rel2.reset_index(drop=True)
                msgid = df3_global_rel['message_id'][0]
                src_link_id = df3_global_rel['src_link_id'][0]
                sent_from = df3_global_rel['From'][0]
                subject = df3_global_rel['Subject'][0]
                date = df3_global_rel['ReceivedDate'][0]
                ae_confidence_score = df3_global_rel['AEConfidenceScore'][0]
                other_confidence_score = df3_global_rel['OtherConfidenceScore'][0]
                global_confidence_score = df3_global_rel['GlobalEmailConfidenceScore'][0]
                df_i = pd.DataFrame()
                df_i.loc[0,'Global_Case_Ids'] = globalid
                df_i.loc[0,'Local_Case_Ids_New'] = None
                df_i.loc[0,'Local_Case_Ids_New_DateTime'] = None
                df_i.loc[0,'Local_Case_Ids_Local'] = None
                df_i.loc[0,'Local_Case_Ids_Local_DateTime'] = None
                df_i.loc[0,'Local_Case_Ids_Global'] = localid
                df_i.loc[0,'Local_Case_Ids_Global_DateTime'] = date
                df_i.loc[0,'message_ID'] = msgid
                df_i.loc[0,'src_link_id'] = src_link_id
                df_i.loc[0,'From'] = sent_from
                df_i.loc[0,'Subject'] = subject
                df_i.loc[0,'Email_Date_Time'] = date
                df_i.loc[0,'AEConfidenceScore'] = ae_confidence_score
                df_i.loc[0,'OtherConfidenceScore'] = other_confidence_score
                df_i.loc[0,'GlobalEmailConfidenceScore'] = global_confidence_score
                df_i.loc[0,'Audit_Flag'] = 'I'
                df_email_global = df_email_global.append(df_i)
    return df_email_global

In [31]:
def local_email_flatten(df3_local):
    
    df_x1 = local_email_df(df3_local)
    if (df_x1.shape[0] == 0):
      df_flatten_temp1 = pd.read_csv(output_path+'Email_Flattened_Temp.csv')
      df_flatten1 = df_flatten_temp1.reset_index(drop=True)
      return df_flatten1
      
    else :
      df_x = df_x1.reset_index(drop=True)
      df_flatten_temp = pd.read_csv(output_path+'Email_Flattened_Temp.csv')
      df_flatten = df_flatten_temp.reset_index(drop=True)
      
      df_final = pd.DataFrame()
      lid_local = df_x['Local_Case_Ids_Local'].unique()
      lid_new = df_flatten['Local_Case_Ids_New'].unique()
      for i in range(len(lid_new)):
          if (lid_new[i] in lid_local):
              df1 = df_x[df_x['Local_Case_Ids_Local'] == lid_new[i]]
              df_final = df_final.append(df1)
          elif (lid_new[i] not in lid_local):
              df2 = df_flatten[df_flatten['Local_Case_Ids_New'] == lid_new[i]]
              df_final = df_final.append(df2)
      for j in range(len(lid_local)):
          if (lid_local[j] not in lid_new):
              df3 = df_x[df_x['Local_Case_Ids_Local'] == lid_local[j]]
              df_final = df_final.append(df3)
              
      df_final.drop(["index"], axis = 1, inplace = True)
      df_final.to_csv(output_path+'Email_Flattened_Temp.csv',index=False)
      return df_final

In [32]:
def global_email_flatten(df3_global):
    
    df_x1 = global_email_df(df3_global) 
    df_x = df_x1.reset_index(drop=True)
    df_flatten_temp = pd.read_csv(output_path+'Email_Flattened_Temp.csv')
    df_flatten = df_flatten_temp.reset_index(drop=True)
    if (df_x1.shape[0] == 0):
      return df_flatten
    
    else:
      df_final = pd.DataFrame()
      lid_global = df_x['Local_Case_Ids_Global'].unique()
      lid_local1 = df_flatten['Local_Case_Ids_Local'].unique()
      lid_new1 = df_flatten['Local_Case_Ids_New'].unique()
      lid_local = [x for x in lid_local1 if str(x) != 'nan']
      lid_new = [x for x in lid_new1 if str(x) != 'nan']
      
      for i in range(len(lid_new)):
          if (lid_new[i] in lid_global):
              df1 = df_x[df_x['Local_Case_Ids_Global'] == lid_new[i]]
              df_final = df_final.append(df1)
          else:
              df2 = df_flatten[df_flatten['Local_Case_Ids_New'] == lid_new[i]]
              df_final = df_final.append(df2)
              
      for k in range(len(lid_local)):
          if (lid_local[k] not in lid_new and lid_local[k] in lid_global):
              df3 = df_x[df_x['Local_Case_Ids_Global'] == lid_local[k]]
              df_final = df_final.append(df3)
          elif (lid_local[k] not in lid_new and lid_local[k] not in lid_global):
              df4 = df_flatten[df_flatten['Local_Case_Ids_Local'] == lid_local[k]]
              df_final = df_final.append(df4)
              
      for j in range(len(lid_global)):
          if (lid_global[j] not in lid_new and lid_global[j] not in lid_local):
              df5 = df_x[df_x['Local_Case_Ids_Global'] == lid_global[j]]
              df_final = df_final.append(df5)
      #df_final.drop(["index"], axis = 1, inplace = True)
      df_final.to_csv(output_path+'Email_Flattened_Temp.csv',index=False)
      return df_final

In [33]:


def oulook_access_authentication():
    # Accessing outlook usering API
    key_val_dict = {}
    key_val_dict = get_keys_from_secret_store_manager()
    access_id = key_val_dict['Application_ID']
    key = key_val_dict['key']
    v_tenant_id = key_val_dict['Tenant_ID']
    credentials = (access_id, key)
    account = Account(credentials, auth_flow_type='credentials', tenant_id=v_tenant_id)
    if account.authenticate():
        print('Authenticated!')
    return account



def move_mail_to_folder(mail,target_folder):
  account = oulook_access_authentication()
  mailbox = account.mailbox(resource=v_resource)
  DSOBOX = mailbox.get_folder(folder_name='COPIED-DSO-CA@takeda.com')
  subfolders_L1 = DSOBOX.get_folders(10)
  for f1 in subfolders_L1:    
    if f1.name == 'TIDE-Folders':
      for f2 in f1.get_folders(10):        
        if f2.name == target_folder:          
          mail.move(f2)
          print('message moved to '+ f2.name)

def fetching_mail_for_a_msgid(msg_id,mail_folder):    
    account = oulook_access_authentication()
    mailbox = account.mailbox(resource=v_resource)
    query = mailbox.new_query().on_attribute('InternetMessageId').contains(msg_id)  # see Query object in Utils
    messages = mailbox.get_messages(query=query,download_attachments=True)    #limit=10,
    for msg in messages:
      #print(msg.object_id)
      print(msg.internet_message_id)
      print(mail_folder)
      move_mail_to_folder(msg,mail_folder)

def email_triage(df_email_list_updated):
    count = int(df_email_list_updated.shape[0])
    for i in range(count):
        msg_id = df_email_list_updated['message_id'][i]
        email_type = df_email_list_updated['email_type'][i]
        if email_type == 'NEW':            
            mail_folder = 'AdverseEvents'
        elif email_type == 'OTHER':
            mail_folder = 'Others'
        elif email_type == 'GLOBAL CASE SENT':
            mail_folder = 'GlobalCaseSent'
        elif email_type == 'LOCAL CASE SENT':
            mail_folder == 'LocalCaseSent'
        elif email_type == 'UNCLASSIFIED':
            mail_folder = 'Unclassified'
        print('*********Msg_Folder_Value******',mail_folder)
        print(msg_id)
        move_message = fetching_mail_for_a_msgid(msg_id,mail_folder)
        print('All messages in the batch have been triaged successfully as per their email types')
        
    
#final_step = email_triage(df_email_list_updated)

        

In [34]:
def get_run_stats(func_name,status):
    t=datetime.datetime.now()                
    if status == 'start':
        task.append(func_name)
        starttime.append(t)
        endtime.append('NA')
        runtime.append('NA')
    elif status == 'end':
        endtime[-1]=t
        t1=starttime[-1]
        t2=endtime[-1]
        t3=(t2-t1).total_seconds()
        runtime[-1]=t3
    data_tuple=list(zip(task,starttime,endtime,runtime))
    df_runtime=pd.DataFrame(data_tuple,columns=['task','starttime','endtime','runtime'])   
    return df_runtime

In [35]:
##########################-------------------MAIN PROGRAM-------------------########################################
####---Batch Level execution -- each dataframe has all the email data of the current ongoing batch


#try:
df_start, batch_id = insert_into_batchlog()

'''
df_batch_detail = audit_batch_log_detail(batch_id,total_emails_count,df_input_to_process,'get_emails_save_attachments_using_validation_sheet_via_windows_api','start')

df_input_to_process,df_validation_sheet_updated = get_emails_save_attachments_using_validation_sheet_via_windows_api(batch_id,validatn_sheet_path,validatn_sheet_name)

total_emails_count = df_input_to_process.shape[0]

df_batch_detail = audit_batch_log_detail(batch_id,total_emails_count,df_input_to_process,'get_emails_save_attachments_using_validation_sheet_via_windows_api','end')
'''


print('*******STARTING READ FLOW*****')
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count,df_input_to_process, 'get_emails_save_attachments_using_validation_sheet_via_windows_api', 'start')
print('*******STARTING READ FLOW1*****')
df_input_to_process = pd.read_csv(curpath + pipeline_input_csv)
print('*******STARTING READ FLOW2*****')
df_input_to_process.reset_index(inplace=True, drop=True)
total_emails_count = df_input_to_process.shape[0]
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count,df_input_to_process, 'get_emails_save_attachments_using_validation_sheet_via_windows_api', 'end')

###df_batch_detail = audit_batch_log_detail(batch_id,total_emails_in_batch,df_input_to_process,'get_emails_save_attachments_within_a_timeframe_using_windows_api_batch','start')
###df_input_to_process = get_emails_save_attachments_within_a_timeframe_using_windows_api_batch(batch_id,Mails_Recieved_After,Time_Window_min)
###total_emails_in_batch = df_input_to_process.shape[0]
###df_batch_detail = audit_batch_log_detail(batch_id,total_emails_in_batch,df_input_to_process,'get_emails_save_attachments_within_a_timeframe_using_windows_api_batch','end')


df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'create_standard_dataframe',
                                         'start')
df_email, df_attch_list = create_standard_dataframe(df_input_to_process)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_email, 'create_standard_dataframe', 'end')

df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'extract_from_attachment',
                                         'start')
df_email_list, df_attachment, df_attch_keyValPair = extract_from_attachment(batch_id, df_email, df_attch_list,
                                                                            output_path)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_email_list, 'extract_from_attachment',
                                         'end')

df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'create_ml_matrix_df', 'start')
df_ml_input = create_ml_matrix_df(batch_id, df_email_list, df_attachment, output_path)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_ml_input, 'create_ml_matrix_df', 'end')

df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'get_Model_output', 'start')
df_ml_output = get_Model_output(df_ml_input)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_ml_output, 'get_Model_output', 'end')

# ********************************************************************************************************************
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'get_mail_types',
                                         'start')
df_email_list_updated = get_mail_types(df_ml_output, df_email_list)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_email_list_updated, 'get_mail_types',
                                         'end')

####--- Start email flattened dataset generation ---####
df1 = pd.read_csv(output_path + 'Email_Type_Final.csv', index_col=False)
df3 = df1[
    ['message_id', 'src_link_id', 'email_type', 'Global_Case_Ids', 'Local_Case_Ids', 'From', 'ReceivedDate', 'Subject',
     'AEConfidenceScore', 'OtherConfidenceScore', 'GlobalEmailConfidenceScore']]
df3_local = df3[df3['email_type'] == 'LOCAL CASE SENT'].drop_duplicates()
df3_global = df3[df3['email_type'] == 'GLOBAL CASE SENT'].drop_duplicates()
df3_new = df3[df3['email_type'] == 'NEW'].drop_duplicates()

## Code block to process 'NEW' email type dataset in Intermediate Dataframe
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'new_email_flatten', 'start')
df_flatten_new = new_email_flatten(df3_new)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_flatten_new, 'new_email_flatten', 'end')

## Code block to process 'LOCAL' email type dataset in Intermediate Dataframe
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'local_email_flatten',
                                         'start')
df_flatten_local = local_email_flatten(df3_local)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_flatten_local, 'local_email_flatten',
                                         'end')

## Code block to process 'GLOBAL' email type dataset in Intermediate Dataframe
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0, 'global_email_flatten',
                                         'start')
df_flatten_global = global_email_flatten(df3_global)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_flatten_global, 'global_email_flatten',
                                         'end')

## Code block to write into email_flattened object
df2 = pd.read_csv(output_path + 'Email_Attachment.csv', index_col=False)
df4 = df2[
    ['attachment_names', 'TradeName', 'GenericName', 'VendorID', 'Reporter', 'PInitial', 'PAge', 'PDob', 'PGender',
     'event_term', 'Narrative', 'src_link_id']]
df_final = pd.merge(df_flatten_global, df4, how='left')
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, 0,
                                         'write_email_flattened_df', 'start')
df_email_flattened = write_email_flattened_df(df_final)
df_batch_detail = audit_batch_log_detail(batch_id, total_emails_count, df_email_flattened,
                                         'write_email_flattened_df', 'end')

# df_batch_detail = audit_batch_log_detail(batch_id,'email_triage','start')
# final_step = email_triage(df_email_list_updated)
# df_batch_detail = audit_batch_log_detail(batch_id,'email_triage','end')

df_end = update_into_batchlog('Completed',total_emails_count)
#except:
    #df_end = update_into_batchlog('Failed',total_emails_count)


Exists!
Exists!
8.0
Audit_Batch_Log entry generated!!!
*******STARTING READ FLOW*****
*******STARTING READ FLOW1*****
*******STARTING READ FLOW2*****
df_initial 905 12
['1__<VI1PR08MB420606F0C839B0A540BB217AA5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-12454-ENT - Entyvio - ADR report - Initial.docx']
1__<VI1PR08MB420606F0C839B0A540BB217AA5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-12454-ENT - Entyvio - ADR report - Initial.docx
['1__<f4a047f92179463a82fe3dc3907d8013@CTSINPUNSXCM006.cts.com>__image001.png', '2__<f4a047f92179463a82fe3dc3907d8013@CTSINPUNSXCM006.cts.com>__image002.jpg']
1__<f4a047f92179463a82fe3dc3907d8013@CTSINPUNSXCM006.cts.com>__image001.png,2__<f4a047f92179463a82fe3dc3907d8013@CTSINPUNSXCM006.cts.com>__image002.jpg
['1__<42de26a15bc8432fa5c134bc2355659a@SG-EXCH01.Innomar.int>__TAKA 17 04633 NIN0543.docx']
1__<42de26a15bc8432fa5c134bc2355659a@SG-EXCH01.Innomar.int>__TAKA 17 04633 NIN0543.docx
['1__<VI1PR08MB4206FF9C9AD05C8BAD134538A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-09705-ENT - Entyvio - ADR report - Initial.docx']
1__<VI1PR08MB4206FF9C9AD05C8BAD134538A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-09705-ENT - Entyvio - ADR report - Initial.docx
['1__<VI1PR08MB420631C5E183BC6D965C8F84A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-10046-ENT - Entyvio - ADR report - Initial.docx']
1__<VI1PR08MB420631C5E183BC6D965C8F84A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-10046-ENT - Entyvio - ADR report - Initial.docx
['1__<VI1PR08MB42067E8C1EDD5A0491B9BDE3A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-10376-ENT - Entyvio - ADR report - Initial.docx']
1__<VI1PR08MB42067E8C1EDD5A0491B9BDE3A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-10376-ENT - Entyvio - ADR report - Initial.docx
['1__<4e45955559fe4c7f9d3e46f04beaa02c@SG-EXCH01.Innomar.int>__TAKA 17 03685 NIN1261.docx']
1__<4e45955559fe4c7f9d3e46f04beaa02c@SG-EXCH01.Innomar.int>__TAKA 17 03685 NIN1261.docx
['1__<6dcb7c0db1f04873a105b5423c5e1b54@SG-EXCH01.Innomar.int>__TAKA 17 05069 NIN1131.docx']
1__<6dcb7c0db1f04873a105b5423c5e1b54@SG-EXCH01.Innomar.int>__TAKA 17 05069 NIN1131.docx
['1__<VI1PR08MB4206BA05099DF572B056B9C1A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-11147-ENT - Entyvio - ADR report - Initial.docx']
1__<VI1PR08MB4206BA05099DF572B056B9C1A5D70@VI1PR08MB4206.eurprd08.prod.outlook.com>__CA-11147-ENT - Entyvio - ADR report - Initial.docx
['1__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image001.png', '2__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image002.jpg', '3__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image003.png', '4__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image004.jpg']
1__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image001.png,2__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image002.jpg,3__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image003.png,4__<DBBPR08MB47893B982482DBEFC83FE17D94940@DBBPR08MB4789.eurprd08.prod.outlook.com>__image004.jpg
['1__<2e1bc1886272426da074de3049c57a9c@DEDUSEXCP001.onetakeda.com>__RKM_C458190612131516.pdf']
1__<2e1bc1886272426da074de3049c57a9c@DEDUSEXCP001.onetakeda.com>__RKM_C458190612131516.pdf
['1__<SG2PR0201MB2157AD03FA00EF8AA23FC3A6FCDD0@SG2PR0201MB2157.apcprd02.prod.outlook.com>__image001.jpg']
1__<SG2PR0201MB2157AD03FA00EF8AA23FC3A6FCDD0@SG2PR0201MB2157.apcprd02.prod.outlook.com>__image001.jpg
['1__<PU1PR02MB25233C7835CEB20C942D83DCC5380@PU1PR02MB2523.apcprd02.prod.outlook.com>__image001.jpg', '2__<PU1PR02MB25233C7835CEB20C942D83DCC5380@PU1PR02MB2523.apcprd02.prod.outlook.com>__image002.png', '3__<PU1PR02MB25233C7835CEB20C942D83DCC5380@PU1PR02MB2523.apcprd02.prod.outlook.com>__image003.png', '4__<PU1PR02MB25233C7835CEB20C942D83DCC5380@PU1PR02MB2523.apcprd02.prod.outlook.com>__image004.png', '5__<

In [36]:

%sh 
pwd
ls
rm *.docx
#/databricks/driver/

/databricks/driver
1__<0342c79ddd9d45978b6a1762e9b3b5be@USLSAEXCP002.onetakeda.com>__Annex G2 LOC Vendor Audit Schedule PSMF Version 25.0 23_Apr_2019.docx
1__<03f38e96dfa3498eba988c491a67fdbb@SG-EXCH01.Innomar.int>__TAKA 17 03804 ALU0043.docx
1__<0911915ff9e24614aec141cae4871a82@SG-EXCH01.Innomar.int>__TAKA 17 04280 NIN1211.docx
1__<0b2e426bb8004df683d17a4d55683bdf@SG-EXCH01.Innomar.int>__TAKA 17 04265 NIN0986.docx
1__<0f41992021ff47fb9aaf00209ae494bc@SG-EXCH01.Innomar.int>__TAKA 17 04429 NIN1427.docx
1__<10eea82bd57840868d9d3377b18f95e6@SG-EXCH01.Innomar.int>__TAKA 17 04989 ALU0055.docx
1__<11833690a9474390825f2af4666b9a32@SG-EXCH01.Innomar.int>__TAKA 17 05135 NIN1437.docx
1__<1299fd8a00e9428bba472d9b454e6bbc@SG-EXCH01.Innomar.int>__TAKA 17 04802 NIN1417.docx
1__<12a6bbee1e394818bf6004ba30329a84@SG-EXCH01.Innomar.int>__TAKA 17 04930 NIN0906.docx
1__<19f52689ef324a6080fb841a74afa990@SG-EXCH01.Innomar.int>__TAKA 17 03948 NIN0765.docx
1__<1c6a0775fab748b298b69bef0f790b7f@SG-EXCH01.Innomar.int>__TAKA 17 04647 NIN0297.docx
1__<22e2aee0156942419b781577087289a9@SG-EXCH01.Innomar.int>__TAKA 17 03682 NIN1275.docx
1__<23d9d3bd7aba4376b4f917bbb1572812@SG-EXCH01.Innomar.int>__TAKA 17 05601 NIN1010.docx
1__<24d189cb0808429893eb1814eb3bc806@SG-EXCH01.Innomar.int>__TAKA 17 04629 NIN1441.docx
1__<25cbd3fd9c094a6483a1ffd8767dba7c@SG-EXCH01.Innomar.int>__TAKA 17 03983 NIN0419.docx
1__<277a23a914b149ce8eef3ca9c94df1e0@SG-EXCH01.Innomar.int>__TAKA 17 03758 NIN0607.docx
1__<2cf1bda7f62b4adda0c2e6a5a3d22894@SG-EXCH01.Innomar.int>__TAKA 17 04231 NIN0714.docx
1__<3202bcd058504dc29d254b6eda350e63@SG-EXCH01.Innomar.int>__TAKA 17 04478 NIN1405.docx
1__<36fa9a327f374d8eadc8a764efce97aa@DEMRHEXCP001.onetakeda.com>__CA-13813- Alunbrig - ADR report-Initial.docx
1__<377188b1a8174cd29fb0cbf5b051ef69@SG-EXCH01.Innomar.int>__TAKA 17 03743 NIN1243.docx
1__<39ebbfbb714940d3ae5cd290127e9b7f@SG-EXCH01.Innomar.int>__TAKA 17 04221 NIN1384.docx
1__<3bf74457dc4246d69643192921e7123f@SG-EXCH01.Innomar.int>__TAKA 17 04219 NIN0680.docx
1__<3e3c1061ada24d13942693d5a99e482e@SG-EXCH01.Innomar.int>__TAKA 17 04036 NIN1236.docx
1__<3ead961d80fb496dad19b20da1868695@SG-EXCH01.Innomar.int>__TAKA 17 05231 NIN1086.docx
1__<3f5944bc8acb4e80a2614277b17ea2e2@DEMRHEXCP001.onetakeda.com>__CA-13126-Ninlaro - ADR Report - Initial.docx
1__<3fe9a01b3daf42b3a691a99e0f5b6558@SG-EXCH01.Innomar.int>__TAKA 17 03786 NIN1208.docx
1__<40516ce21ed847c7b7ad98e87f77bc55@SG-EXCH01.Innomar.int>__TAKA 17 04246 NIN1408.docx
1__<40fc02b291bc4039833f1eecb478f79d@SG-EXCH01.Innomar.int>__TAKA 17 04951 NIN0841.docx
1__<42de26a15bc8432fa5c134bc2355659a@SG-EXCH01.Innomar.int>__TAKA 17 04633 NIN0543.docx
1__<444a3e7310444ec493e58e47a4dd7e13@SG-EXCH01.Innomar.int>__TAKA 17 04153 NIN0557.docx
1__<44adf1bc71e8457c93a80d9a33a46dca@SG-EXCH01.Innomar.int>__TAKA 17 05168 NIN0030.docx
1__<4771139548d743c7b650c73a089a620e@SG-EXCH01.Innomar.int>__TAKA 17 05393 NIN1351.docx
1__<47ed14c998e347ffa1f48002ebb83959@SG-EXCH01.Innomar.int>__TAKA 17 04460 NIN1312.docx
1__<4bb4383a69414248887605ed21297506@SG-EXCH01.Innomar.int>__TAKA 17 05142 NIN1232.docx
1__<4c39d29940184ea28c83b2b32f2a1a4f@SG-EXCH01.Innomar.int>__TAKA 17 04473 ALU0040.docx
1__<4c62a20fd3d449a099d83c0065deb03a@SG-EXCH02.Innomar.int>__TAKA 17 03889 NIN1350.docx
1__<4dd44aa18d614ee2b40fd4550cc9ad4b@SG-EXCH01.Innomar.int>__TAKA 17 03926 NIN0023.docx
1__<4e45955559fe4c7f9d3e46f04beaa02c@SG-EXCH01.Innomar.int>__TAKA 17 03685 NIN1261.docx
1__<50ec6fd21e8b45c18b2c8b3119fda464@SG-EXCH01.Innomar.int>__TAKA 17 04720 NIN1454.docx
1__<518ee4e3de594acda16c03ac9a3b8f37@SG-EXCH01.Innomar.int>__TAKA 17 04527 NIN0951.docx
1__<51ed3176e5ab4effb34c71938b10c678@SG-EXCH01.Innomar.int>__TAKA 17 04750 NIN1432.docx
1__<54553f0f73464d54b69f741acc409807@SG-EXCH01.Innomar.int>__TAKA 17 04731 NIN1461.docx
1__<552a8743390e43a2a20c40221db2c7ea@SG-EXCH01.Innomar.int>__TAKA 17 04691 ALU0061.docx
1__<5537f1a0abbf4761a56858cf892cd332@SG-EXCH01.Innomar.int>__TAKA 17 05151 NIN0405.docx
1__<5546f2b798f